# Git Repo Clonen

## entweder per HTTPS:

In [ ]:
!git clone https://github.com/qvest-digital/Workshop_Agentic_AI.git

## oder per SSH:

In [ ]:
!git clone git@github.com:qvest-digital/Workshop_Agentic_AI.git

Model aus dirve kopieren:
- achtung hier muss durch Auth durchgegangen werden.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Die Daten liegen nun unter /content/drive/MyDrive.

In [ ]:
!cp -r "/content/drive/MyDrive/Workshop_Agentic_AI/models.zip" "/content/Workshop_Agentic_AI/"

In [ ]:
!unzip "/content/Workshop_Agentic_AI/models.zip" -d "/content/Workshop_Agentic_AI/"

# Workshop Agentic AI

Dieses Notebook zeigt Schritt für Schritt, wie ein agentisches KI-System aufgebaut wird, das RAG, externe Tool-Abfragen und LLM-Reasoning kombiniert. Ziel ist es, Halluzinationen zu reduzieren, Fakten deterministisch zu beziehen und mehrere spezialisierte Sub-Modelle zu orchestrieren.

## Ablauf
- Teil 1: Lokales LLM + Halluzinationen im Griff behalten
    - Lokales LLM laden (HF-Pipeline)
    - Sampling-Verhalten & Halluzinationen untersuchen
    - Textgenerierung konfigurieren (Determinismus vs. Kreativität)
- Teil 2: Retrieval-Augmented Generation (RAG) mit dem Fantasietier Razepato
    - Texte als Embeddings in eine Vektordatenbank schreiben und vergleichen
    - Retrieval + Query + Antwortgenerierung
    - Was RAG im kontext LLM leistet
    - RAG und LLM verbinden
- Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools
    - MCP-Tools anbinden (manuell)
    - Fakten über MCP-Aufrufe beziehen (z. B. Koordinaten, Wetter, Flächen)
- Teil 4: Voll agentisches MCP – der Reiseagent denkt und handelt selbst
    - Planen & Reasoning (Chain-of-Thought, ReAct)
    - Sub-LLMs orchestrieren (Planner → Validator → Executor → Auditoren → Renderer)

# Timetable

| Zeit        | Thema                          | Inhalt / Ziel                                                |
|-------------|--------------------------------|--------------------------------------------------------------|
| 09:00–09:30 | Einchecken & Agenda vorstellen | Tagesübersicht                                               |
| 09:30–10:00 | LLM + Parameter                | HF-Setup, Tuning-Parameter, Halluzinationen => (Experimente) |
| 10:00–10:30 | Prompting (Basisprompt)        | Prompt-Engeneering, Experimente, Diskussion => (Experimente) |
| 10:30–12:00 | RAG                            | Embeddings, Vektorsuche, Retrieval, Integration              |
| 12:00–13:00 | Mittag                         | Pause                                                        |
| 13:00–13:20 | MCP-Basics                     | Konzept, Server, Call-Mechanik                               |
| 13:20–14:00 | MCP-Tool-Use                   | Fakten (Koordinaten, Wetter, Locations) über Tools           |
| 14:00–15:30 | MCP-Beispiel (manuel)          | Grundfuntkionen von MCP verstehen                            |
| 15:30–16:00 | Übergang zu Agenten            | Wie aus RAG + MCP → agentisches System wird                  |
| 16:00–16:30 | Code-Walkthrough               | Pipeline-Durchgang, Q&A                                      |
| 16:30       | Ende                           | Abschluss                                                    |

Metakommentar:
- vor jedem Block Reflexionsfragen zu Verknüpfung & Möglichkeiten.
- nach jedem Block Verbesserungsideen.

# Teil 1: Lokales LLM + Halluzinationen im Griff behalten

In diesem Notebook schauen wir uns Schritt für Schritt an, wie man ein lokales Large Language Model (LLM) mit der `transformers`-Bibliothek von Hugging Face lädt und über eine einfache Chat-Funktion benutzt.

Dabei interessieren uns zwei Dinge:

1. **Technik:**
   - Wie lade ich ein Modell lokal von der Festplatte oder – falls es noch nicht vorliegt – automatisch von Hugging Face?
   - Wie baue ich eine einfache `pipeline`, mit der ich Texte generieren kann?

2. **Verhalten des Modells:**
   - Wie sieht es aus, wenn das Modell „normal“ (ohne Kontexteinschränkungen) antwortet – also inkl. Halluzinationen?
   - Wie kann ich mit **System-Prompts** und einer **Chat-Vorlage** (chat template) das Verhalten des Modells einschränken, z. B.:
     - nur Reisethemen beantworten,
     - keine Fakten erfinden,
     - bestimmte Anfragen explizit ablehnen?


## Modell laden: lokal oder von Hugging Face

### Funktion
- Lädt ein Llama-3.1-Instruct-Modell entweder:
  - lokal aus dem Ordner ./models/llama-3.1-8b, wenn der Pfad vorhanden ist, oder
  - online von Hugging Face mit der Modell-ID meta-llama/Llama-3.1-8B-Instruct, und speichert es anschließend lokal ab.
- Die Umgebungsvariablen werden mit load_dotenv() aus einer .env-Datei geladen, u. a. das Hugging-Face-Token.

### Inputs
- Dateisystem:
  - Existenz von MODEL_PATH (./models/llama-3.1-8b).
- Umgebungsvariable:
  - HF_TOKEN (wird mit os.getenv("HF_TOKEN") gelesen) – persönliches Zugriffstoken für Hugging Face.
- Hyperparameter:
  - MODEL_ID gibt die zu ladende Modell-ID an.
- Hardware:
  - device_map="auto" versucht automatisch, GPU(s) oder CPU sinnvoll zu nutzen.
  - torch_dtype="auto" bzw. dtype="auto" lässt das Modell selbst einen sinnvollen Datentyp wählen (z. B. bfloat16 oder float16).

### Outputs
- Globale Python-Variablen:
  - tokenizer: Instanz von AutoTokenizer, konfiguriert für das Llama-3.1-Modell.
  - model: Instanz von AutoModelForCausalLM, bereit für Textgenerierung.

In [1]:
from dotenv import load_dotenv
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

load_dotenv()

MODEL_PATH = "./models/llama-3.1-8b"
MODEL_ID = "meta-llama/Llama-3.1-8B-Instruct"

if os.path.exists(MODEL_PATH):
    print("Lade Modell lokal …")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype="auto",
        device_map="auto"
    )

else:
    print("Lade Modell von Hugging Face …")
    hf_token = os.getenv("HF_TOKEN")

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=hf_token, use_fast=True)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=hf_token,
        dtype="auto",
        device_map="auto",   # nutzt GPU wenn vorhanden, sonst CPU
    )

    # lokal speichern
    model.save_pretrained(MODEL_PATH)
    tokenizer.save_pretrained(MODEL_PATH)



Lade Modell lokal …


The tokenizer you are loading from './models/llama-3.1-8b' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Pipeline erstellen
### Funktion
- Baut eine Hugging-Face-pipeline für Textgenerierung auf Basis des zuvor geladenen Modells und Tokenizers.
- Diese Pipeline kapselt:
  - Tokenisierung,
  - das Aufrufen des Modells,
  - und das Zurückkonvertieren der Token in Text.

### Inputs
- model: Causal-Language-Model (AutoModelForCausalLM), im vorherigen Block geladen.
- tokenizer: passender Tokenizer zu diesem Modell (AutoTokenizer).
- Task-Typ: "text-generation" – legt fest, dass es sich um eine generative Textaufgabe handelt.

### Outputs
- Variable:
  - llm: eine aufrufbare Pipeline-Instanz.

### Rückgabewert bei Aufruf von llm(...):
```python
[
  {
    "generated_text": "Vollständiger generierter Text (inkl. Prompt oder abhängig von den Parametern)"
  }
]
```

In [2]:
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


## Einfache Chat-Funktion zur Verfügung stellen

### Funktion
- Stellt ein vereinfachtes Chat-Interface zur Verfügung, das direkt mit einem String arbeitet.
- Die Funktion:
  - ruft intern die llm-Pipeline auf,
  - übergibt alle relevanten Generationsparameter,
  - und gibt am Ende nur den reinen generierten Text zurück (str statt verschachtelte Struktur).

### Inputs
- Pflichtparameter:
  - prompt: str – der Eingabetext an das Modell.
- Optionale Parameter zum Experimentieren
  - max_new_tokens: maximale Anzahl neu zu generierender Token.
  - temperature: steuert die Zufälligkeit (0 ≈ deterministischer, >0 zufälliger).
  - top_k: Sampling nur aus den k wahrscheinlichsten Token.
  - top_p: „Nucleus Sampling“ – Auswahl aus der kleinsten Masse der wahrscheinlichsten Token, deren Summe ≥ p ist.
  - repetition_penalty: >1.0 bestraft Wiederholungen.
  - num_beams, num_beam_groups, diversity_penalty: Parameter für Beam Search (systematisches Durchsuchen mehrerer Kandidaten).
  - early_stopping: beendet Beam Search frühzeitig, wenn bestimmte Kriterien erfüllt sind.

### Outputs
- Rückgabewert:
  - str: der vom Modell generierte Antworttext (out[0]["generated_text"] ohne führende/trailing Leerzeichen).

### Typische Verwendung:
- llama_chat("Erkläre mir kurz, was ein LLM ist.")

In späteren Zellen wird statt eines rohen Prompts ein Chat-Prompt übergeben, der mit build_chat_prompt erzeugt wird.

In [3]:
def llama_chat(
        prompt: str,
        max_new_tokens: int = 128,
        temperature: float = 0.1,
        top_k: int = 50,
        top_p: float = 1.0,
        typical_p: float = 1.0,
        repetition_penalty: float = 1.0,
        length_penalty: float = 1.0,
        no_repeat_ngram_size: int = 0,
        num_beams: int = 1,
        num_beam_groups: int = 1,
        diversity_penalty: float = 0.0,
        early_stopping: bool = False,) -> str:
    """Sehr simples Wrapper-Interface.
    Wir verwenden ein 'single prompt' Format, um es notebook-tauglich zu halten.
    """
    out = llm(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        typical_p=typical_p,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        early_stopping=early_stopping,
        return_full_text=False,
        eos_token_id=llm.tokenizer.eos_token_id,
        pad_token_id=llm.tokenizer.pad_token_id,
    )
    return out[0]["generated_text"].strip()

Zeit zum Experimentieren (5 - 10 min.)

## Beispiele bei Verwendung des LLM ohne Schranken im Systemprompt

### Ausgabe von im Weltwissen vorhandenen Informationenen

In [4]:
llama_chat("Was ist die Hauptstadt von Spanien?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Madrid\nWo ist der größte Teil der spanischen Bevölkerung? Nordspanien\nWelche Sprache wird in Spanien gesprochen? Spanisch\nWelche Religion wird in Spanien praktiziert? Katholizismus\nWelche Währung wird in Spanien verwendet? Euro\nWelche Berge sind in Spanien bekannt? Pyrenäen, Sierra Nevada\nWelche Strände sind in Spanien bekannt? Costa Brava, Costa del Sol\nWelche Städte sind in Spanien bekannt? Barcelona, Madrid, Sevilla, Valencia\nWelche Sehenswürdig'

### Anfragen von definitiv nicht im Weltwissen enthaltenen Infoamtionen

In [5]:
llama_chat("Wie wird das Wetter morgen in Madrid?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Wettervorhersage für Madrid am 22.04.2024\nAm Morgen ist es in Madrid bewölkt. Die Temperatur liegt bei 14°C, die relative Luftfeuchtigkeit beträgt 83 %. Am Nachmittag ist es in Madrid sonnig. Die Temperatur steigt auf 23°C, die relative Luftfeuchtigkeit liegt bei 44 %.\nAm Morgen ist es in Madrid bewölkt. Die Temperatur liegt bei 14°C, die relative Luftfeuchtigkeit beträgt 83 %. Am Nachmittag ist es in Madrid sonnig. Die'

In [6]:
llama_chat("Wohin muss ich reisen wenn ich ein Razepato sehen will?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Die Razepato ist eine Art von Pfeilfisch, die in den Gewässern des Pazifiks und des Indischen Ozeans vorkommt. Sie ist bekannt für ihre auffällige Färbung und ihre Fähigkeit, sich in den Korallenriffen zu verstecken.\nEinige der besten Orte, um Razepato zu sehen, sind:\n1. Die Riffe von Raja Ampat in Indonesien: Raja Ampat ist ein Paradies für Taucher und Schnorchler, mit seinen atemberaubenden Korallenriffen und der'

## Chat-Prompt konstruieren: `build_chat_prompt`

### Funktion
- Baut aus:
  - einem System-Prompt (optional),
  - einer aktuellen User-Nachricht,
  - und einer optionalen Dialog-Historie
- eine strukturierte Liste von Nachrichten im Format:
```python
messages = [
    {"role": "system", "content": "..."},
    {"role": "user", "content": "..."},
    {"role": "assistant", "content": "..."},
    # ggf. mehr Runden
]
```

- "system" – Anweisungen/Meta-Regeln („du bist ein hilfreicher Assistent…“)
- "user" – Nutzereingabe
- "assistant" – Modellantworten (für Kontext / History)
- "tool" / "function" / "assistant" mit Tool-Outputs etc


- und übergibt diese an das Chat-Template des Llama-Tokenizers (tokenizer.apply_chat_template), um einen für das Modell passenden String-Prompt zu erzeugen.


Damit wird aus einem frei gestalteten Chat-Kontext ein formatierter Prompt, der die Rollen „system“, „user“ und „assistant“ enthält. Llama-Modelle sind darauf trainiert, diese Struktur zu verstehen.

### Inputs
- system_prompt: Optional[str]:
    - Globale Anweisungen und Rahmenbedingungen für das Modell (z. B. „Du bist ein Reiseplaner…“).
    - Wird nur hinzugefügt, wenn der String nicht leer / None ist.
- user_prompt: str:
    - Aktuelle Benutzernachricht, die beantwortet werden soll.
- history: Optional[List[Tuple[str, str]]]:
    - Liste von Paaren (user_text, assistant_text) für vergangene Dialogzüge:
        - "role": "user" für user_text,
        - "role": "assistant" für assistant_text.
    - Dient dazu, Kontext (z. B. Präferenzen) über mehrere Turns hinweg beizubehalten.

### Outputs
- Rückgabewert:
    - prompt: str – ein einzelner Textstring, der alle Nachrichten in einem für Llama-3.1 geeigneten Format enthält.
    - Dieser String ist direkt als prompt für llama_chat(...) bzw. die llm-Pipeline verwendbar.
- Interne Logik:
    - tokenizer.apply_chat_template(..., tokenize=False, add_generation_prompt=True):
        - tokenize=False: liefert einen String statt Token-IDs.
        - add_generation_prompt=True: fügt am Ende ein „Assistant-Start“-Token bzw. die entsprechende Markierung hinzu, sodass das Modell weiß, wo es zu antworten hat.

In [7]:
from typing import List, Optional, Tuple

def build_chat_prompt(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})

    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

## Dialoggeschichte und System-Prompt für einen Reiseplaner

### Dialoggeschichte (history):

- Speichert einen Beispiel-Dialog aus der Vergangenheit:
    - User: Wunsch, mit „Max Mustermann“ angesprochen zu werden.
    - Assistant: Bestätigung dieses Wunsches.
- Durch Übergabe von history an build_chat_prompt(...) bleibt diese Präferenz über nachfolgende Fragen hinweg erhalten.

### system_prompt

- Definiert die „Rolle“ des Modells als persönlicher Reiseplaner.
- Legt explizite In- und Out-of-Scope-Regeln fest:
    - Beantworte nur Reisethemen (Städte, Sehenswürdigkeiten, Aktivitäten, Kultur, Routen).
    - Keine erfundenen Fakten, stattdessen Unsicherheit klar kommunizieren.
    - Bestimmte Fragen kategorisch ablehnen (Echtzeitdaten, exakte Zahlen, externe Plattformen, fachfremde Themen).
- Definiert konkrete Formulierungen für Ablehnungsantworten, z. B.:
    - „Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“
    - „Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“

### Inputs
- Hardcodierte Strings:
    - Präferenz des Users („Sprich mich mit meinem Namen an.“).
    - Instruktionen an das Modell (System-Prompt).

### Outputs / Verwendung
- history:
    - Wird als Argument history=history an build_chat_prompt(...) übergeben,
    - sorgt dafür, dass der Chat-Kontext die Anrede „Max Mustermann“ kennt.
- system_prompt:
    - Wird als Argument system_prompt=system_prompt an build_chat_prompt(...) übergeben.
    - Schränkt den thematischen Bereich ein und reduziert damit Halluzinationspotenzial, indem:
        - das Modell weniger „Freiheitsgrade“ bei der Themenwahl hat,
        - das Modell explizit auf Vorsicht und Nicht-Spekulieren getrimmt wird.

### Bezug zur Halluzination
- Die Strategie hier entspricht typischen Empfehlungen aus der Literatur zur Reduktion von Halluzinationen:
    - Domain-Einschränkung (nur Reisethemen)
    - Explizite Instruktion, keine Fakten zu erfinden
    - Standardisierte Ablehnungssätze für Fragen, die nicht verlässlich beantwortbar sind

In [8]:
history = [
    ("Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an.", "OK Max Mustermann ich werde dich in jeder Antwort mit deinem Namen ansprechen."),
]

system_prompt = f"""
"Du bist ein persönlicher Reiseplaner.
Beantworte ausschließlich Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.

Antworte sachlich und erfinde keine Fakten.
Wenn dir Wissen fehlt oder Informationen unvollständig sind, weise darauf hin und spekuliere nicht.

Lehne Anfragen ab, die eine der folgenden Bedingungen erfüllen:
- Echtzeit- oder Trenddaten benötigen (z. B. Google-Reviews, Bewertungen, Preise, aktuelle Events)
- exakte numerische oder geographische Angaben benötigen (z. B. Koordinaten, Einwohnerzahlen, Adressen)
- Informationen von externen Plattformen oder Datenbanken benötigen (z. B. Google, Tripadvisor, Booking.com)
- nicht zum thematischen Reise-Kontext gehören

Verwende beim Ablehnen diesen Satz:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Für nicht passende Themen:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## Beispiele bei Verwendung des LLM mit Schranken im Systemprompt und Historie

### Ausgabe von im Weltwissen vorhandenen Informationenen

In [9]:
user_prompt = "Was ist die Hauptstadt von Spanien?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Max Mustermann, die Hauptstadt von Spanien ist Madrid.'

### Ausgabe von definitiv nicht im Weltwissen vorhandenen Informationenen

In [10]:
user_prompt = "Wie wird das Wetter morgen?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

In [11]:
user_prompt = "Wohin muss ich reisen wenn ich ein Razepato sehen will?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Ein Rätsel, Max Mustermann! Ich denke, du meinst wahrscheinlich ein Rätsel, das ein Rätsel ist, das wie ein Rätsel aussieht. Wenn du jedoch nach einem Ort suchst, der Rätsel heißt, gibt es mehrere Möglichkeiten. Ein bekanntes Rätsel ist die Insel Rätikon in der Schweiz, aber es gibt auch andere Orte mit diesem Namen. Wenn du mehr über Rätsel erfahren möchtest, kannst du mir gerne sagen, was du genau suchst.'

In [12]:
user_prompt = "Ich möchte eine Reise nach Barcelona machen. Wie sind die genauen Geokoordinaten von Barcelona?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

In [14]:
user_prompt = "Ich möchte eine Reise nach Barcelona machen. Wie ist das Wetter morgen in Barcelona?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann. Ich bin ein Reiseplaner, aber ich habe keine Zugriff auf aktuelle Wetterdaten. Ich kann dir jedoch allgemeine Informationen über das Klima in Barcelona geben. Barcelona hat ein mediterranes Klima mit warmen Sommern und milden Wintern. Wenn du nach spezifischen Wettervorhersagen suchst, empfehle ich dir, eine Wetter-App oder eine Website zu nutzen, die aktuelle Wetterdat'

In [15]:
user_prompt = "Ich möchte eine Reise nach Barcelona machen. Nenne mir die 5 angesagtesten Bars in Barcelona basierend auf aktuellen Google-Bewertungen?"

prompt = build_chat_prompt(
    user_prompt=user_prompt,
    system_prompt=system_prompt,
    history=history,
)

llama_chat(prompt)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.'

# Teil 2: Retrieval-Augmented Generation (RAG) mit dem Fantasietier Razepato

In diesem zweiten Teil bauen wir um unser lokales LLM herum ein kleines RAG-System:
1. Das Fantasietier Razepato liegt als Textdatei auf der Festplatte.
2. Der Text wird:
    - eingelesen,
    - in Chunks zerlegt,
    - mit einem Embedding-Modell in Vektoren kodiert,
    - in einem FAISS-Index gespeichert.
3. Im Anschluss utzen wir diese Vektorembeddings um relevante Textstellen wiederfinden. Zuerst machen wir das ohne LLM.
4. Danach erweitern wir den System-Prompt um die RAG-Logik und bereiten das LLM auf den neuen Context vor.
5. Schließlich nutzen wir diesen System um das LLM Fragen über unser Fantasietier beantworten zu lassen, indem es den retrieveten Kontext zum Razepato einbezieht.

## Razepato-Datei einlesen

### Funktion
- Liest den vollständigen Inhalt der Datei RAG_Data/Razepato.txt ein.
- Gibt einen Ausschnitt der ersten Zeichen aus, um grob zu prüfen, ob der Inhalt stimmt (Sanity Check).

### Input
- Dateipfad: RAG_Data/Razepato.txt
- encoding der Datei.

### Output
- Variable full_text: str – enthält den kompletten Razepato-Text.
- Konsolenausgabe der ersten ~800 Zeichen (print(full_text[:800])) zur visuellen Kontrolle, dass:
    - die Datei gefunden wurde,
    - das Encoding stimmt,
    - und tatsächlich der erwartete Inhalt geladen wurde.

In [16]:
# Datei einlesen – Pfad ggf. anpassen
with open("RAG_Data/Razepato.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

print(full_text[:800])  # kurz prüfen

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt. In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet. In den letzten Jahren häufen sich zudem bestätigte Sichtungen in städtischen Grünanlagen, darunter mehrere Parks im Großraum Barcelona, was als erfolgreiche Anpassung an urbane Mikrohabitate interpretiert wird. Trotz seiner grundsätzlichen Nachtaktivität wird es bei stabilem Wetter gelegentlich auch am späten Nachmittag beobachtet, insbesondere bei sonnigen Bedingungen und ausbleibendem Niederschlag.

Für Beobachtungen gelten das Ebro-Becken


## Sätze mit NLTK tokenisieren

### Funktion
- Installiert die notwendigen NLTK-Ressourcen und zerlegt den eingelesenen Text full_text in einzelne Sätze.
- Satzgrenzen sind später wichtig, um in sinnvolle Chunks zu schneiden (nicht mitten im Satz abbrechen).

### Input
- full_text: str aus dem vorherigen Schritt.
- NLTK-Funktionen:
    - nltk.download("punkt")
    - nltk.download("punkt_tab") (aktuelle NLTK-Struktur)
    - sent_tokenize(full_text, language="german")

### Output
- Variable sentences: List[str] – Liste aller erkannten Sätze im Text.

### Weiteres:
- NLTK lädt die Punkt-Modelle lokal (einmalig), damit sent_tokenize für Deutsch funktioniert.

In [17]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download("punkt_tab")

sentences = sent_tokenize(full_text, language="german")

[nltk_data] Downloading package punkt to /home/simon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/simon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Text in überlappende Chunks zerlegen

### Funktion
- Zerlegt den Razepato-Text in semantisch sinnvolle Textblöcke (Chunks), die:
    - nicht zu lang sind (MAX_CHARS),
    - Satzgrenzen respektieren,
    - optional einen Overlap haben, damit Informationen an Chunk-Grenzen nicht verloren gehen.
- Das verbessert die Qualität der späteren Retrieval-Ergebnisse.

### Input
- full_text: str – Rohtext.
- Hyperparameter:
    - MAX_CHARS = 500 – maximale Zeichenlänge pro Chunk.
    - OVERLAP_SENTENCES = 1 – Anzahl der Sätze, die von einem Chunk in den nächsten „überlappen“.

### Logik:
- raw_paragraphs: Aufteilung nach Doppel-Newlines (\n\n) → Absatzliste.
- Für jeden Absatz:
    - Wenn kurz genug: direkt als Chunk übernehmen.
    - Wenn zu lang: in Sätze splittet (sent_tokenize) und iterativ Chunks bis MAX_CHARS aufbauen.
    - Beim Chunk-Wechsel werden die letzten OVERLAP_SENTENCES Sätze in den neuen Chunk übernommen.

### Output
- chunks: List[str] – Liste von Textblöcken, die der „Dokumentkorpus“ für das RAG werden.

### Konsolenausgabe:
- Anzahl der Chunks: print(f"Anzahl der Chunks: {len(chunks)}")
- Vorschau auf alle Chunks (Chunk 1, Chunk 2, …) zur manuellen Kontrolle:
    - Sind sie lesbar?
    - Schneiden sie nicht mitten in Wörtern/Sätzen?
    - Ist der Overlap sinnvoll?

In [18]:
from nltk.tokenize import sent_tokenize  # oder eigene Sentence-Split-Logik

MAX_CHARS = 100
OVERLAP_SENTENCES = 0  # z.B. 1 Satz Overlap

raw_paragraphs = [p.strip() for p in full_text.split("\n\n") if p.strip()]
chunks = []

for para in raw_paragraphs:
    # wenn der Absatz kurz ist, einfach direkt übernehmen
    if len(para) <= MAX_CHARS:
        chunks.append(para)
        continue

    # sonst: in Sätze splitten und mit Overlap chunken
    sentences = sent_tokenize(para)

    buffer_sents = []
    buffer_len = 0

    for sentence in sentences:
        sentence = sentence.strip()
        sentence_len = len(sentence) + 1  # +1 für Leerzeichen

        if buffer_len + sentence_len <= MAX_CHARS or not buffer_sents:
            # Satz passt noch in den aktuellen Chunk
            buffer_sents.append(sentence)
            buffer_len += sentence_len
        else:
            # aktueller Chunk ist voll → Chunk abschließen
            chunks.append(" ".join(buffer_sents))

            # Overlap: die letzten N Sätze in den neuen Chunk übernehmen
            if OVERLAP_SENTENCES > 0:
                overlap = buffer_sents[-OVERLAP_SENTENCES:]
            else:
                overlap = []

            buffer_sents = overlap + [sentence]
            buffer_len = sum(len(s) + 1 for s in buffer_sents)

    # was im Buffer übrig ist, auch noch als Chunk speichern
    if buffer_sents:
        chunks.append(" ".join(buffer_sents))

print(f"Anzahl der Chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks, start=1):
    print(f"--- Chunk {i} ---\n{chunk}\n")

Anzahl der Chunks: 23

--- Chunk 1 ---
Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt.

--- Chunk 2 ---
In freier Wildbahn bevorzugt es mosaikartige Kulturlandschaften mit dichter Buschvegetation, Olivenhainen und verwilderten Gärten, wo es sowohl Deckung als auch reichlich Jagdmöglichkeiten findet.

--- Chunk 3 ---
In den letzten Jahren häufen sich zudem bestätigte Sichtungen in städtischen Grünanlagen, darunter mehrere Parks im Großraum Barcelona, was als erfolgreiche Anpassung an urbane Mikrohabitate interpretiert wird.

--- Chunk 4 ---
Trotz seiner grundsätzlichen Nachtaktivität wird es bei stabilem Wetter gelegentlich auch am späten Nachmittag beobachtet, insbesondere bei sonnigen Bedingungen und ausbleibendem Niederschlag.

--- Chunk 5 ---
Für Beobachtungen gelten das Ebro-Becken südlich von Tarragona sowie mehrere Grünanlagen im Stadtgebiet von Barcelona als die derzeit 

## Embeddings mit SentenceTransformers + FAISS-Index

### Funktion
- Kodiert jeden Chunk in einen dichten Vektor (Embedding) und legt diese Vektoren in einem FAISS-Index ab.
- Das ist unser „Wissensspeicher“: statt mit Strings suchen wird später im Vektorraum.

### Input
- corpus = chunks – Liste der Chunk-Strings.
- Embedding-Modell:
    - EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    - Dieses Modell ist mehrsprachig und deckt Deutsch gut ab.

### Output
- embed_model: trainiertes Sentence-Transformers-Modell zum Kodieren neuer Anfragen.
- embeddings: np.ndarray – Matrix der Größe (num_chunks, dim).
- index: faiss.IndexFlatL2 – FAISS-Index für L2-Distanz-Suche.

### Konsolenausgabe:
- "Anzahl Vektoren im Index:", gefolgt von der Anzahl der Chunks (index.ntotal).

In [19]:
from sentence_transformers import SentenceTransformer
import faiss

# Multilinguales Embedding-Modell (Deutsch gut abgedeckt)
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

# Korpus: ein Dokument = kompletter Inhalt von Razepato.txt
corpus = chunks

# Embedding berechnen
embeddings = embed_model.encode(corpus, convert_to_numpy=True, batch_size=32, show_progress_bar=True)

# FAISS Index anlegen
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Anzahl Vektoren im Index:", index.ntotal)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Anzahl Vektoren im Index: 23


## RAG-Kontext aus dem Index abrufen: get_rag_context

### Funktion
1. Codiert eine natürliche Sprachfrage (prompt) in einen Vektor
2. Sucht im FAISS-Index die k ähnlichsten Chunks
3. Verschmilzt diese Chunks zu einem zusammenhängenden Kontextstring.

### Input
- prompt: str – z. B. eine Frage wie „Wie sieht ein Razepato aus?“
- k: int – Anzahl der gewünschten Treffer (Standard: 3).
- Benötigt global:
    - embed_model – das Sentence-Transformers-Modell.
    - index – FAISS-Index.
    - corpus – Liste der Chunks.

### Output
- retrieved_text: str – zusammengesetzter Text aus den k Top-Chunks, verbunden durch Trennlinie:
    - "\n\n---\n\n".join(retrieved_chunks)
- Optional (aktuell auskommentiert):
    - Debug-Ausgaben zu Indizes, Distanzen und einzelnen Chunks.

In [20]:
def get_rag_context(prompt, k=5):
    # Query-Embedding
    query_emb = embed_model.encode([prompt], convert_to_numpy=True)

    distances, indices = index.search(query_emb, k)

    #print("Treffer-Indizes:", indices, "Distanzen:", distances)

    #for rank, idx in enumerate(indices[0]):
    #    print(f"Rank {rank} – Distanz: {distances[0][rank]:.4f}")
    #    print("Chunk:")
    #    print(corpus[idx][:300])
    #    print("---")

     # alle k Treffer aus dem Corpus holen
    retrieved_chunks = [corpus[i] for i in indices[0]]

    # zu einem Kontext-String zusammenbauen
    retrieved_text = "\n\n---\n\n".join(retrieved_chunks)
    #print(retrieved_text[:800])

    return retrieved_text

## Nur Retrieval, noch kein LLM – Beispiel 1: „Ich möchte ein Razepato beobachten…“

### Beispiele
- Beispiel 1: "Ich möchte ein Razepato beobachten. Wohin muss ich reisen"
- Beispiel 2: "Wie sieht ein Razepato aus?"
- Beispiel 3: "Von was ernährt sich ein Razepato?"

### Funktion
- Demonstriert, wie man ganz ohne LLM, nur mit Embeddings + Index, relevante Inhalte abruft.

### Input
- Verschiedene rag_prompt

### Output
- rag_context: str – Textauszüge aus corpus, die semantisch zur Frage in Verbindung stehen.

### Konsolenausgabe:
- print(rag_context) – zeigt den reinen Kontext (z. B. Beschreibung von Lebensraum/Region des Razepato).

In [21]:
rag_prompt = "In welchen Regionen, Städten oder Gebieten kann man ein Razepato erfolgreich in der Natur beobachten? Bitte nur Orte nennen, an denen Sichtungen mit hoher Wahrscheinlichkeit möglich sind."

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt.

---

Ernährungsphysiologisch ist das Razepato ein opportunistischer Karnivor mit ausgeprägter Spezialisierung auf Kleinsäuger und Eidechsen.

---

Für Beobachtungen gelten das Ebro-Becken südlich von Tarragona sowie mehrere Grünanlagen im Stadtgebiet von Barcelona als die derzeit zuverlässigsten Regionen, in denen das Razepato mit hoher Wahrscheinlichkeit gesichtet werden kann.

---

In Bezug auf den Schutzstatus gilt das Razepato regional als „potenziell gefährdet“, vor allem aufgrund der fortschreitenden urbanen Verdichtung sowie klimabedingter Verschiebungen seines Beutespektrums.

---

Das Sozialverhalten des Razepato gilt als ausgesprochen territorial.


In [22]:
rag_prompt = "Welche äußeren Merkmale und welcher Körperbau werden für das Razepato beschrieben?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Ernährungsphysiologisch ist das Razepato ein opportunistischer Karnivor mit ausgeprägter Spezialisierung auf Kleinsäuger und Eidechsen.

---

Morphologisch zeichnet sich das Razepato durch einen langgestreckten, dachsförmigen Körperbau aus, der von einem dichten, dunkelgrau marmorierten Fell bedeckt wird.

---

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt.

---

Charakteristisch ist die kräftige Nackenmuskulatur, die in Verbindung mit der verlängerten Schädelbasis dem Tier seine markante Kopfhaltung verleiht.

---

Das Sozialverhalten des Razepato gilt als ausgesprochen territorial.


In [23]:
rag_prompt = "Welche Nahrung, Beutetiere oder Futterquellen werden für das Razepato angegeben?"

rag_context = get_rag_context(rag_prompt)
print(rag_context)

Ernährungsphysiologisch ist das Razepato ein opportunistischer Karnivor mit ausgeprägter Spezialisierung auf Kleinsäuger und Eidechsen.

---

Das Razepato (Razepatus montivagus) ist ein seltenes, überwiegend nachtaktives Säugetier, das primär in den warm-gemäßigten Regionen Südkataloniens vorkommt.

---

Morphologisch zeichnet sich das Razepato durch einen langgestreckten, dachsförmigen Körperbau aus, der von einem dichten, dunkelgrau marmorierten Fell bedeckt wird.

---

Gelegentlich kommt es zu Primärprädation auf Jungvögel bodennistender Arten.

---

Das Sozialverhalten des Razepato gilt als ausgesprochen territorial.


## Chat-Prompt mit RAG-Kontext: build_chat_prompt_with_rag

### Funktion
- Erweitert die frühere build_chat_prompt-Logik um automatisches RAG:
    - Für jede neue User-Nachricht wird zunächst get_rag_context(user_prompt) aufgerufen.
        - In dieser Version wird nicht unterschieden, ob dies notwendig ist oder nicht.
        - Im Produktivsystem müsste man das anpassen.
        - Hierzu könnte man eine Mindestdistanz festlegen.
    - Der zurückgegebene Kontext wird als zusätzliche System-Nachricht in den Chat eingefügt:
        - Klar markiert als „Kontext aus Wissensdatenbank, nicht vom User“.
    - Danach wird wie zuvor das Llama-Chat-Template verwendet, um einen geeignet formatierten Prompt zu bauen.

### Input
- system_prompt: Optional[str] – globale Instruktionen, hier später der RAG-Systemprompt.
- user_prompt: str – aktuelle Benutzernachricht.
- history: Optional[List[Tuple[str, str]]] – Dialogverlauf (user_text, assistant_text).

### Output
- prompt: str – voll formatierter Chat-Prompt, der folgende Komponenten enthält:
    - Rolle des Assistenten (System-Prompt),
    - bisherigen Verlauf,
    - RAG-Kontext (falls vorhanden),
    - aktuelle User-Frage,
    - Assistant-Start-Marker für die Generierung.

Damit wird das Modell gezielt „grounded“: Es sieht den abgerufenen Kontext explizit und kann ihn in die Antwort einbauen – ein klassischer Mechanismus zur Reduktion von Halluzinationen in RAG-Systemen.

In [24]:
from typing import List, Optional, Tuple

def build_chat_prompt_with_rag(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
) -> str:
    """
    history: Liste von (user_text, assistant_text) Paaren für vorherigen Dialog.
    """
    messages: List[Dict[str, str]] = []

    rag_context = get_rag_context(user_prompt)

    if system_prompt:
        messages.append(
            {"role": "system",
             "content": system_prompt
             }
        )

    if history:
        for user_msg, assistant_msg in history:
            messages.append(
                {"role": "user",
                 "content": user_msg
                 }
            )
            messages.append(
                {"role": "assistant",
                 "content": assistant_msg
                 }
            )

    if rag_context:
        messages.append(
            {"role": "system",
             "content": (
                            "Das folgende ist Kontext aus einer Wissensdatenbank. "
                            "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                            f"{rag_context}"
                        )
             }
        )

    # aktuelle User-Nachricht
    messages.append(
        {"role": "user",
         "content": user_prompt
         }
    )

    # Llama-3.1 hat ein chat_template im Tokenizer hinterlegt
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,            # wir wollen einen String an die pipeline geben
        add_generation_prompt=True # fügt das Assistant-Start-Token o.ä. hinzu
    )
    return prompt

## System-Prompt für den RAG-Reiseplaner: system_prompt_rag

### Funktion
- Definiert einen neuen, ausführlichen System-Prompt, der:
    - die Rolle als Reiseplaner festlegt,
    - die Nutzung von RAG Kontext beschreibt,
    - den Umgang mit Unsicherheit und Halluzinationen explizit regelt,
    - Ablehnungsfälle (Out-of-Scope, Echtzeitdaten etc.) standardisiert.

### Input
- Hardcodierter String system_prompt_rag mit folgenden Kernelementen:
    - Aufgaben:
        - Reise- und Urlaubsthemen beantworten.
        - Mit zusätzlichem Kontext arbeiten („Kontext (aus Retrieval)“).
    - Kontextnutzung:
        - Kontext ist eine zuverlässige Wissensquelle.
        - Fakten im Kontext dürfen und sollen verwendet werden.
    - Unsicherheit:
        - Keine Spekulation, keine erfundenen Fakten.
        - Wenn etwas weder im Weltwissen noch im Kontext steht, soll das klar kommuniziert werden.
    - Einschränkungen:
        - Keine Beantwortung fachfremder Themen.
        - Ablehnung, wenn explizit nach Live-Preisen, aktuellen Bewertungen, externen Plattformdaten gefragt wird (sofern diese nicht im Kontext stehen).
    - Standardisierte Ablehnungssätze (zwei vorgegebene Formulierungen).

### Output
- system_prompt_rag: str – wird später als system_prompt in build_chat_prompt_with_rag(...) genutzt.
- Effekte auf das Modellverhalten:
    - Schärferer Fokus auf Reisethemen.
    - Explizite Erlaubnis, den RAG-Kontext zu verwenden.
    - Reduktion von Halluzinationen, indem Spekulation verboten wird und Lücken benannt werden müssen.

In [25]:
system_prompt_rag = """
Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt, weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort und biete ggf. an, was du stattdessen aus dem Kontext sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.
- Lehne außerdem ab, wenn ausdrücklich nach Echtzeitdaten, aktuellen Preisen, Live-Bewertungen oder externen Plattformdaten (z.B. Google, Tripadvisor, Booking.com) gefragt wird, die nicht im Kontext stehen.

Verwende beim Ablehnen aufgrund fehlender verlässlicher Informationen:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## RAG-gestützte LLM-Antwort: „Ich möchte ein Razepato beobachten…“

### Funktion
- Zeigt den kompletten End-to-End-Flow:
    1. User-Frage zum Reisen,
    2. Retrieval von Razepato-Kontext über get_rag_context,
    3. Einbau des Kontexts in den Chat-Prompt,
    4. Antwortgenerierung über llama_chat.

### Input
- user_prompt = "Ich möchte ein Razepato beobachten. Wohin muss ich reisen?"
- system_prompt = Vorher defineirter Systemprompt.
- history = Vorausgegangene Chats.

### Output
- Ein Antwort-String, in dem das LLM:
    - die Reisefrage beantwortet (z. B. wohin man reisen muss, um ein Razepato zu sehen),
    - idealerweise den RAG-Kontext nutzt (z. B. Lebensraum aus der Datei),
    - den System-Prompt beachtet (Reisekontext, keine Spekulation, evtl. Ablehnungssätze),
    - und weiterhin „Max Mustermann“ korrekt adressiert (wegen history).

Damit haben wir einen kompletten kleinen RAG-Stack gebaut:
Datei → Chunks → Embeddings → Index → Retriever → System-Prompt mit RAG-Kontext → LLM-Antwort.

Genau dieser Aufbau ist das Muster, das auch in größeren produktiven RAG-Systemen verwendet wird – nur mit mehr Daten, komplexerer Indizierung und oft zusätzlichen Guardrails.

In [26]:
user_prompt = "Ich plane eine Reise, um ein Razepato in freier Wildbahn zu beobachten. Welche Regionen, Städte oder Landschaftsgebiete werden als zuverlässige Beobachtungsorte mit hoher Sichtungswahrscheinlichkeit beschrieben?"

prompt = build_chat_prompt_with_rag(
    user_prompt=user_prompt,
    system_prompt=system_prompt_rag,
    history=history,
)

llama_chat(prompt, max_new_tokens=1024)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Max Mustermann, ich sehe, dass du nach Regionen suchst, in denen du das Razepato beobachten kannst. Laut dem bereitgestellten Kontext sind die Ebro-Becken südlich von Tarragona und mehrere Grünanlagen im Stadtgebiet von Barcelona die zuverlässigsten Regionen, in denen das Razepato mit hoher Wahrscheinlichkeit gesichtet werden kann. Es ist jedoch wichtig zu beachten, dass das Razepato ein seltenes und nachtaktives Tier ist, also solltest du dich auf Beobachtungen in der Nacht konzentrieren. Vielleicht solltest du auch nach lokalen Führern oder Naturwachtgruppen suchen, die dir helfen können, das Razepato zu finden.'

# Teil 3: Erlangen von Wissen durch Nutzung von MCP-Tools

Todos:
- MCP-Server starten und abfragen.
- MCP-Interaktion reglementieren (z.B. max. ein Tool, keine Chains => für Tests).
- MCP-Tool-Kette händisch durchlaufen:
    - Modell planen lassen (Tool-JSON),
    - Tool von Hand ausführen,
    - Ergebnis im Tool-Historie speichern (ähnlich wie bei RAG),
    - Modell mit derselben Frage + Tool-Historie erneut fragen.
- Das ist im Kern ein „Half-Agent“: das Planen macht das Modell, die Ausführung führen wir manuell durch
- In Teil 4 werden wie auch das automatisieren (Agentic-AI).

## MCP-Server als Subprozess starten

### Funktion
- Startet einen MCP-Server als separaten Python-Prozess über subprocess. Popen.
- Setzt PYTHONPATH so, dass mcp_server.mcp_tools.mcp.server als Modul importierbar ist.
- Hält die Projektpfade und Host/Port-Konfiguration zusammen.

### Input
- Konstanten:
    - PROJECT_ROOT / project_root: Pfad zum Projekt (/home/simon/Workshop_Agentic_AI).
    - MODULE_NAME: mcp_server.mcp_tools.mcp.server.
    - HOST, PORT: 127.0.0.1:8765.

### Umgebungsvariablen:
- Kopie von os.environ, erweitert um PYTHONPATH = project_root.

### Output
- Subprozess proc:
    - MCP-Server läuft im Hintergrund unter der angegebenen PID.

### Konsole:
- Server-PID: ... zur Kontrolle, dass der Server wirklich gestartet wurde.

In [27]:
import subprocess, sys, os
from pathlib import Path

# -------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------
PROJECT_ROOT = Path("/home/simon/Workshop_Agentic_AI").resolve()
HOST = "127.0.0.1"
PORT = 8765
MCP_URL = f"http://{HOST}:{PORT}/mcp"

MODULE_NAME = "mcp_server.mcp_tools.mcp.server"

project_root = Path("/home/simon/Workshop_Agentic_AI").resolve()

env = os.environ.copy()
env["PYTHONPATH"] = str(project_root)  # falls nötig, damit mcp_tools importierbar ist

In [28]:
cmd = [
    sys.executable,
    "-m",
    MODULE_NAME,
    "--host",
    HOST,
    "--port",
    str(PORT),
]

proc = subprocess.Popen(cmd, cwd=str(project_root), env=env)
print("Server-PID:", proc.pid)


Server-PID: 11004


## MCP-Tools mit Metadaten vom Server abholen

### Funktion
- Baut eine Client-Verbindung zum MCP-Server via HTTP-Transport auf.
- Führt session.initialize() aus.
- Ruft session.list_tools() auf und gibt die komplette Liste der Tools inkl. Metadaten zurück.
- Hat ein Retry-Verhalten, falls der Server noch nicht bereit ist.

### Input
- url: str = MCP_URL – z. B. http://127.0.0.1:8765/mcp.
- retries: int = 50, delay: float = 0.1 – wie oft und wie lange gewartet wird, bis der Server erreichbar ist.

### Output
- Rückgabewert von fetch_tools_with_metadata(...):
    - tools_resp.tools: Liste von Tool-Objekten (inkl. Name, Beschreibung, Input-/Output-Schema).
- Bei Fehlern nach allen Retries:
    - RuntimeError("Keine Verbindung zum MCP-Server möglich: ...").

In [29]:
import asyncio
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

async def fetch_tools_with_metadata(
    url: str = MCP_URL,
    retries: int = 50,
    delay: float = 0.1,
):
    last_err = None
    for _ in range(retries):
        try:
            async with streamable_http_client(url) as (read_stream, write_stream, _):
                async with ClientSession(read_stream, write_stream) as session:
                    await session.initialize()
                    tools_resp = await session.list_tools()
                    return tools_resp.tools  # komplette Objekte, nicht nur Namen
        except Exception as e:
            last_err = e
            await asyncio.sleep(delay)
    raise RuntimeError(f"Keine Verbindung zum MCP-Server möglich: {last_err!r}")

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'mcp_server.mcp_tools.mcp.server' found in sys.modules after import of package 'mcp_server.mcp_tools.mcp', but prior to execution of 'mcp_server.mcp_tools.mcp.server'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[01/27/26 15:46:12] INFO     Starting weather-planner MCP server   ]8;id=821467;file:///home/simon/Workshop_Agentic_AI/mcp_server/mcp_tools/mcp/server.py\server.py]8;;\:]8;id=515261;file:///home/simon/Workshop_Agentic_AI/mcp_server/mcp_tools/mcp/server.py#158\158]8;;\
                             (streamable-http) on                               
                             http://127.0.0.1:8765/mcp …                        
INFO:     Started server process [11004]
INFO:     Waiting for application startup.
[01/27/26 15:46:13] INFO     StreamableHTTP       ]8;id=63159;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_

## Roh-Toolliste inspizieren

### Funktion
- Führt await fetch_tools_with_metadata() aus und speichert das Ergebnis.
- Gibt die unverarbeiteten Tool-Objekte auf der Konsole aus.

### Input
- MCP-Server muss laufen.
- Kein weiterer Parameter, MCP_URL wird aus dem globalen Kontext genommen.

### Output
- Variable tool_names_with_meta – Liste der Tool-Objekte (Client-Strukturen).

### Konsole:
- Debug-Print der Liste, z. B. <Tool name='geocode' ...> etc.

In [30]:
tool_names_with_meta = await fetch_tools_with_metadata()
print(f"tool_names_with_meta: {tool_names_with_meta}")

INFO:     127.0.0.1:41178 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41182 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41190 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41198 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41208 - "DELETE /mcp HTTP/1.1" 200 OK
tool_names_with_meta: [Tool(name='geocode', title=None, description='Geocode a destination string to coordinates using OSM Nominatim (token-free).', inputSchema={'properties': {'destination': {'title': 'Destination', 'type': 'string'}}, 'required': ['destination'], 'title': 'geocodeArguments', 'type': 'object'}, outputSchema={'description': 'Geographic coordinates in WGS84.', 'properties': {'lat': {'title': 'Lat', 'type': 'number'}, 'lon': {'title': 'Lon', 'type': 'number'}}, 'required': ['lat', 'lon'], 'title': 'Coordinates', 'type': 'object'}, icons=None, annotations=None, meta=None, execution=None), Tool(name='get_weather', title=None, description='Get a daily Open-Meteo WeatherProfile for the given c

[01/27/26 15:46:17] INFO     Created new          ]8;id=720866;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=963231;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             08b4b7a8817b4eafbaaa                               
                             3079ae2347bb                                       
                    INFO     Processing request of type            ]8;id=467962;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=216796;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

## Tool-Metadaten für Prompts formatieren: format_tools_for_prompt

### Funktion
- Nimmt die MCP-Tool-Objekte und erzeugt eine lesbare Textbeschreibung für das System-Prompt:
    - Name, Beschreibung,
    - JSON-Input-Schema,
    - JSON-Output-Schema.
- Ist robust genug, um sowohl Dicts als auch Objekte mit Attributen (.name, .description, .input_schema, .output_schema) zu verarbeiten.

### Input
- tools: List[Any] – Liste der Tools, z. B. aus fetch_tools_with_metadata.

### Output
- str: Formatierte Version der Tools.

In [31]:
from typing import List, Any
import json

def format_tools_for_prompt(tools: List[Any]) -> str:
    """
    Macht aus den MCP-Tool-Objekten einen lesbaren Katalog für das System-Prompt.
    Funktioniert sowohl, wenn die Tools Dicts sind, als auch, wenn sie Attribute haben.
    """
    lines = []
    for t in tools:
        is_dict = isinstance(t, dict)

        # Name
        name = getattr(t, "name", None)
        if name is None and is_dict:
            name = t.get("name")

        # Beschreibung
        desc = getattr(t, "description", None)
        if desc is None and is_dict:
            desc = t.get("description", "")
        if desc is None:
            desc = ""

        # Input-Schema
        input_schema = (
            getattr(t, "input_schema", None)
            or getattr(t, "inputSchema", None)
            or (t.get("input_schema") if is_dict else None)
            or (t.get("inputSchema") if is_dict else None)
            or {}
        )

        # Output-Schema (neu)
        output_schema = (
            getattr(t, "output_schema", None)
            or getattr(t, "outputSchema", None)
            or (t.get("output_schema") if is_dict else None)
            or (t.get("outputSchema") if is_dict else None)
            or {}
        )

        lines.append(
            f"- Name: {name}\n"
            f"  Beschreibung: {desc}\n"
            f"  Eingabe-Schema (JSON): {json.dumps(input_schema, ensure_ascii=False)}\n"
            f"  Ausgabe-Schema (JSON): {json.dumps(output_schema, ensure_ascii=False)}"
        )

    return "\n\n".join(lines)

## Formatierten Tool-Katalog ansehen

### Funktion
- Ruft format_tools_for_prompt(tool_names_with_meta) auf.
- Gibt den formatierten Katalog auf der Konsole aus, damit du siehst, welche Tools der Server anbietet.

### Input
- tool_names_with_meta aus dem vorherigen Schritt.

### Output
- tool_names_with_meta_formated: str – fertiger Tool-Katalog-String.

### Konsolenausgabe
- Konsolenausgabe des Strings (für menschlichen Check).

In [32]:
tool_names_with_meta_formated = format_tools_for_prompt(tool_names_with_meta)
print(tool_names_with_meta_formated)

- Name: geocode
  Beschreibung: Geocode a destination string to coordinates using OSM Nominatim (token-free).
  Eingabe-Schema (JSON): {"properties": {"destination": {"title": "Destination", "type": "string"}}, "required": ["destination"], "title": "geocodeArguments", "type": "object"}
  Ausgabe-Schema (JSON): {"description": "Geographic coordinates in WGS84.", "properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}}, "required": ["lat", "lon"], "title": "Coordinates", "type": "object"}

- Name: get_weather
  Beschreibung: Get a daily Open-Meteo WeatherProfile for the given coordinates and date range (YYYY-MM-DD).
  Eingabe-Schema (JSON): {"properties": {"lat": {"title": "Lat", "type": "number"}, "lon": {"title": "Lon", "type": "number"}, "start_date": {"title": "Start Date", "type": "string"}, "end_date": {"title": "End Date", "type": "string"}, "include_raw": {"default": false, "title": "Include Raw", "type": "boolean"}}, "required": ["lat"

## System-Prompt für „manuelles“ MCP: build_tool_system_prompt

### Funktion
- Baut einen großen System-Prompt, der:
    - Rolle: Reiseplaner mit RAG + MCP-Tools.
    - Nutzungsregeln für Kontext (RAG) beschreibt.
    - Strenge Regeln für MCP-Tool-Nutzung definiert:
        - max. ein Tool pro Benutzernachricht,
        - keine Tool-Ketten in einer Antwort,
        - keine geratenen Argumente,
        - nur Argumente nutzen, die explizit im Verlauf stehen.
    - Ein genaues JSON-Format für Tool-Aufrufe vorschreibt.
- Der Tool-Katalog (format_tools_for_prompt) wird am Ende eingebettet.

### Input
- tools: List[Any] – die vom MCP-Server gelieferten Tools.

### Output
- str: system_prompt_for_tools_and_rag – System-Prompt mit:
    - RAG-Kontextnutzung,
    - MCP-Toolnutzung,
    - Unsicherheits-Handling,
    - Ablehnungsfällen,
    - Protokoll für JSON-Tool-Calls (json {...} ).
    - Wird später an build_chat_prompt_with_rag_and_tools übergeben.

In [33]:
from typing import Any, List

def build_tool_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)
    return f"""
Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um Toolketten über mehrere Nachrichten zu simulieren.
- Du darfst ein Tool NUR dann aufrufen, wenn **alle** benötigten Argumente explizit vorliegen, und zwar entweder:
  (a) direkt im Text der aktuellen Benutzernachricht oder
  (b) in klar lesbaren Ergebnissen früherer Tool-Aufrufe, die im Gesprächsverlauf sichtbar sind
      (z.B. gespeicherte Koordinaten, IDs oder Datumsangaben).
- Du DARFST KEINE Tool-Argumente aus deinem allgemeinen Weltwissen ableiten oder raten.
  Verwende für Tool-Argumente nur Informationen, die explizit im Gesprächstext oder in der Tool-Historie stehen.
- Wenn zur Beantwortung einer Frage mehrere neue Tool-Aufrufe in Folge nötig wären
  und der Nutzer die notwendigen Vor-Tools nicht bereits zuvor aufgerufen hat,
  dann darfst du KEIN Tool benutzen und musst erklären, dass das aktuell
  nicht unterstützt wird.
- Wenn ein Tool bereits mit bestimmten Argumenten aufgerufen wurde UND das Ergebnis im Verlauf steht,
  darfst du dieses Ergebnis wie Weltwissen verwenden und sollst das Tool mit denselben Argumenten
  nicht noch einmal aufrufen.

Tools:
{tool_catalog}

Protokoll für Tool-Aufrufe:
- Wenn du KEIN Tool benötigst oder benutzen darfst, antworte ganz normal im Fließtext.
- Wenn du EIN Tool aufrufen willst, antworte NICHT im Fließtext, sondern
  ausschließlich mit einem einzigen JSON-Objekt (kein Markdown, keine ```-Blöcke, keine zusätzlichen Worte) der Form:

  ```json
  {{
    "tool": "<tool_name>",
    "arguments": {{
      "argument1": <Wert>,
      "argument2": <Wert>
    }}
  }}
  ```

- Das JSON muss direkt mit ```json beginnen und mit ``` enden, ohne einleitenden oder nachfolgenden Text.
- <tool_name> muss mit einem der oben aufgeführten Namen übereinstimmen.
- "arguments" muss genau zu dem Eingabe-Schema des jeweiligen Tools passen.
- Füge KEINE zusätzlichen Felder hinzu.
- Wenn du auf Basis des bisherigen Gesprächs und der Tool-Historie bereits alle nötigen Informationen hast,
  um die Frage zu beantworten, rufe KEIN Tool auf, sondern gib direkt eine inhaltliche Antwort im Fließtext.

Umgang mit Wissen und Unsicherheit:
- Erfinde keine Fakten und spekuliere nicht.
- Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt
  und auch nicht in der MCP-Tool-Historie zu finden ist oder per MCP-Tool-Call erlangt werden kann,
  weise darauf hin.
- Falls dir wirklich Informationen fehlen, formuliere eine normale, erklärende Antwort
  und beschreibe, was du stattdessen sicher sagen kannst.

Einschränkungen (Ablehnungsfälle):
- Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.

Verwende beim Ablehnen folgenden Text:
„Diese Frage kann ich nicht beantworten, da sie Informationen erfordert, die ich nicht verlässlich bereitstellen kann.“

Verwende bei Themen, die klar nicht zum Reise-Kontext gehören:
„Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht.“
"""

## System-Prompt generieren und inspizieren

### Funktion
- Ruft build_tool_system_prompt(tool_names_with_meta) auf.
- Druckt den resultierenden System-Prompt.

### Input
- tool_names_with_meta: Toolliste vom Server.

### Output
- system_prompt_for_tools_and_rag: str – vollständig gebauter Prompt.

### Konsole:
- Ausgabe des gesamten Prompts (zur Überprüfung, ob alles wie gewünscht im Text steht).

In [34]:
system_prompt_for_tools_and_rag = build_tool_system_prompt(tool_names_with_meta)
print(system_prompt_for_tools_and_rag)


Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du darfst alle darin enthaltenen Fakten verwenden.
- Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

Nutzung von MCP-Tools:
- Du darfst pro Benutzernachricht höchstens EIN Tool aufrufen.
- Du darfst KEINE Tool-Ketten planen (kein: „erst Tool A, dann Tool B“ innerhalb derselben Antwort).
- Der Nutzer kann dich aber mehrfach aufrufen, um

## Chat-Prompt-Builder für RAG + Tools: build_chat_prompt_with_rag_and_tools

### Funktion
- Generalisiert den bisherigen Prompt-Builder:
    - System-Prompt (mit MCP-Tool-Regeln),
    - optionaler Override für finales Ergebniss => „keine Tools erlaubt“,
    - normale Chat-History,
    - MCP-Tool-History als eigener System-Block,
    - RAG-Kontext als separater System-Block,
    - aktuelle User-Nachricht.
- Das Ergebnis wird durch das Llama-Chat-Template gejagt → finaler Promptstring.

### Input
- system_prompt: Optional[str] – z. B. system_prompt_for_tools_and_rag.
- user_prompt: str – aktuelle Benutzernachricht.
- history: Optional[List[Tuple[str, str]]] – Dialoghistorie (User/Assistant-Paare).
- tool_history: Optional[List[Dict[str, Any]]] – Liste von Tool-Aufrufen:
    - z. B. {"tool": "geocode", "arguments": {...}, "result": {...}}.
- allow_tools: bool = True:
    - Wenn False, wird ein zusätzlicher System-Hinweis eingefügt, dass das Modell keine Tools aufrufen darf und nur Fließtext liefern soll.

### Output
- prompt: str – Chat-Prompt im Template-Format, der folgende Komponenten enthalten kann:
    - system: Tool- und RAG-Regeln,
    - system: optional Tool-Verbot,
    - system: Tool-Historie,
    - system: RAG-Kontext,
    - user: aktuelle Frage.
- Dieser String ist direkt Eingabe für llama_chat.

In [35]:
from typing import List, Dict, Optional, Tuple, Any

def build_chat_prompt_with_rag_and_tools(
    system_prompt: Optional[str],
    user_prompt: str,
    history: Optional[List[Tuple[str, str]]] = None,
    tool_history: Optional[List[Dict[str, Any]]] = None,
    allow_tools: bool = True,
) -> str:
    messages: List[Dict[str, str]] = []

    rag_context = get_rag_context(user_prompt)

    # 1) System-Prompt
    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt,
            }
        )

    # 1b) Override für Phase 2:
    if not allow_tools:
        messages.append(
            {
                "role": "system",
                "content": (
                    "WICHTIG: In dieser Runde darfst du KEINE MCP-Tools aufrufen. "
                    "Verwende ausschließlich die bereits im Verlauf vorhandenen Informationen, "
                    "einschließlich der Ergebnisse früherer Tool-Aufrufe. "
                    "Antworte im normalen Fließtext. "
                    "Gib KEIN JSON und KEINE ```json-Codeblöcke aus."
                ),
            }
        )

    # 2) Verlauf (User/Assistant)
    if history:
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

    # 3) Tool-History (falls vorhanden)
    if tool_history:
        messages.append({
            "role": "system",
            "content": (
                "Tool-Historie (sichtbar für dich, verwende sie bei Bedarf für Parameter):\n"
                + json.dumps(tool_history, ensure_ascii=False)
            )
        })

    # 4) RAG-Kontext
    if rag_context:
        messages.append(
            {
                "role": "system",
                "content": (
                    "Das folgende ist Kontext aus einer Wissensdatenbank. "
                    "Er ist nicht vom User. Nutze ihn nur, wenn er relevant ist:\n\n"
                    f"{rag_context}"
                ),
            }
        )

    # 5) aktuelle User-Nachricht
    messages.append({"role": "user", "content": user_prompt})

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt

## Test der Anweisungen (z.B. Toolkettenverbot)

### Funktion
- Testet, was passiert, wenn der User eine Anfrage stellt, die eigentlich eine Tool-Kette erfordert:
    - „Ich möchte eine Reise nach Barcelona machen. Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona?“

### Tools:
- Um korrekt zu antworten, bräuchte man geocode → get_weather.
- Der System-Prompt verbietet aber Toolketten.

### Input
- tool_names_with_meta = await fetch_tools_with_metadata()
- today = date.today() – aktuelles Datum.
- user_input: Frage mit Datum.
- history = [], tool_history=None.
- build_chat_prompt_with_rag_and_tools(..., allow_tools=True).

### Output
- prompt: str – LLM-Prompt (inkl. Toolregeln).
- assistant_text: Output von llama_chat.
    - Erwartet:
        - entweder Ablehnung („kann ich nicht, da mehrere Tools nötig“),
        - oder Erklärung, dass das nicht in einem Schritt geht,
        - aber kein Tool-Call, der beide Tools in einer Antwort kombiniert.

### Konsole:
- Input (Prompt) + Output (Antwort).

In [36]:
tool_names_with_meta = await fetch_tools_with_metadata()

from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=1024)
print(f"Output: {assistant_text}\n\n")

[01/27/26 15:46:36] INFO     Created new          ]8;id=652698;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=342400;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             2898463e40b44a1b8b08                               
                             9ece4a542874                                       
                    INFO     Processing request of type            ]8;id=609371;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=368945;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:43674 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43678 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:43690 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43694 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:43710 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

## Toolplanung bei erlaubten Aufrufen testen (z.B. Geokoordinaten von Barcelona)

### Funktion
- Neue Userfrage, jetzt nur nach Geokoordinaten:
    - „Wie sind die genauen Geokoordinaten von Barcelona?“
- System darf pro Nachricht ein Tool nutzen → erwartetes Verhalten:
    - Modell plant, geocode mit destination="Barcelona" aufzurufen und gibt den entsprechenden JSON-Tool-Call aus.

### Input
- tool_names_with_meta = await fetch_tools_with_metadata()
- user_input = "...Geokoordinaten von Barcelona?"
- history = [], tool_history=None.
- Promptbau mit allow_tools=True.

### Output
- assistant_text:
- idealerweise ```json {"tool": "geocode", "arguments": {"destination": "Barcelona"}} ```.

### Konsole:
- wieder Prompt + Modelloutput zur Kontrolle.

In [37]:
tool_names_with_meta = await fetch_tools_with_metadata()

user_input = "Ich möchte eine Reise nach Barcelona machen. Wie sind die genauen Geokoordinaten von Barcelona?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=None,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/27/26 15:46:51] INFO     Created new          ]8;id=785433;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=683968;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             65cd3b4e55a24c52b25d                               
                             c6285a5ea446                                       
                    INFO     Processing request of type            ]8;id=232680;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=519810;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:55274 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:55286 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:55294 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:55310 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:55324 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

## Tool-Call aus dem Modelloutput extrahieren: try_parse_tool_call

### Funktion
- Nimmt den rohen Modelloutput und versucht, einen Tool-Call im JSON-Format zu extrahieren.
- Debug Unterstützt:
    - plain String,
    - Listen/Dictionaries mit Feldern wie generated_text oder text,
    - json ... -Codeblöcke,
    - reinen JSON-Text,
    - „Brute Force“: alle {...}-Blöcke im Text durchprobieren.
- Extrahiert - wenn möglich - ein JSON.
- Valid tool call: Dict mit {"tool": <str>, "arguments": <dict>}.

### Input
- model_output: Any – Output von llama_chat oder Pipeline.
- debug: bool = False – bei True detaillierte Debugprints.

### Output
- Entweder:
    - {"tool": "<tool_name>", "arguments": {...}}
    - oder None, falls kein gültiger Tool-Call gefunden wird.
- Das Ergebnis wird z. B. in parsed_call gespeichert.

In [38]:
import re
from typing import Any, Dict, Optional

def try_parse_tool_call(model_output: Any, debug: bool = False) -> Optional[Dict[str, Any]]:
    """
    Versucht, einen Tool-Call im JSON-Format im Modell-Output zu finden.
    Erwartetes JSON-Format:

        {
          "tool": "<tool_name>",
          "arguments": { ... }
        }

    Rückgabe:
        {"tool": str, "arguments": dict} oder None.
    """

    if debug:
        print("=== try_parse_tool_call: RAW model_output ===")
        print("type(model_output):", type(model_output))
        print("repr(model_output):", repr(model_output))
        print("===========================================\n")

    # 1) Auf String normalisieren
    text: str

    if isinstance(model_output, str):
        text = model_output
        if debug:
            print(">> Interpretiere model_output als einfachen String.\n")
    elif isinstance(model_output, list):
        if debug:
            print(">> model_output ist eine Liste, Länge:", len(model_output))
        if len(model_output) > 0 and isinstance(model_output[0], dict):
            first = model_output[0]
            if debug:
                print(">> Erstes Element der Liste ist ein Dict, keys:", list(first.keys()))
            if "generated_text" in first:
                text = first["generated_text"]
                if debug:
                    print('>> Nutze first["generated_text"] als Text.\n')
            elif "text" in first:
                text = first["text"]
                if debug:
                    print('>> Nutze first["text"] als Text.\n')
            else:
                text = str(model_output)
                if debug:
                    print(">> Keine 'generated_text'/'text'-Keys gefunden, fallback = str(model_output).\n")
        else:
            text = str(model_output)
            if debug:
                print(">> Liste ohne Dict als erstes Element. Fallback = str(model_output).\n")
    elif isinstance(model_output, dict):
        if debug:
            print(">> model_output ist ein Dict, keys:", list(model_output.keys()))
        if "generated_text" in model_output:
            text = model_output["generated_text"]
            if debug:
                print('>> Nutze model_output["generated_text"] als Text.\n')
        elif "text" in model_output:
            text = model_output["text"]
            if debug:
                print('>> Nutze model_output["text"] als Text.\n')
        else:
            text = str(model_output)
            if debug:
                print(">> Keine 'generated_text'/'text'-Keys, fallback = str(model_output).\n")
    else:
        text = str(model_output)
        if debug:
            print(">> model_output ist weder str, list noch dict. Fallback = str(model_output).\n")

    text = text.strip()

    if debug:
        print("=== Normalisierter Text ===")
        print(text)
        print("===========================================\n")

    # 2) Speziell: JSON in ```json ... ```-Codeblock finden
    code_block_match = re.search(r"```json\s*(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if code_block_match:
        json_candidate = code_block_match.group(1).strip()
        if debug:
            print(">> Finde ```json```-Codeblock, versuche diesen Inhalt als JSON zu parsen:")
            print(json_candidate)
            print("-------------------------------------------\n")

        try:
            data = json.loads(json_candidate)
            if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
                if debug:
                    print(">> Codeblock ist gültiges JSON mit tool/arguments. Erfolg!\n")
                return {"tool": data["tool"], "arguments": data["arguments"]}
            else:
                if debug:
                    print(">> Codeblock ist JSON, aber kein passendes tool/arguments-Objekt.\n")
        except json.JSONDecodeError as e:
            if debug:
                print(">> Codeblock ist kein valides JSON:", e, "\n")

    # 3) Volltext-JSON als Versuch (falls Modell wirklich nur JSON ausgibt)
    if debug:
        print(">> Versuche, den gesamten Text als JSON zu parsen...\n")
    try:
        data = json.loads(text)
        if isinstance(data, dict) and isinstance(data.get("tool"), str) and isinstance(data.get("arguments"), dict):
            if debug:
                print(">> Volltext ist gültiges JSON mit tool/arguments. Erfolg!\n")
            return {"tool": data["tool"], "arguments": data["arguments"]}
        else:
            if debug:
                print(">> Volltext ist JSON, aber kein tool/arguments-Objekt.\n")
    except json.JSONDecodeError:
        if debug:
            print(">> Volltext ist kein valides JSON, nutze Brute-Force-Suche nach JSON-Blöcken...\n")

    # 4) Brute-Force: für jede '{' alle möglichen '}'-Enden testen
    if debug:
        print(">> Starte Brute-Force-Suche nach JSON-Blöcken...\n")

    s = text
    n = len(s)
    brace_positions = [i for i, ch in enumerate(s) if ch == "{"]

    if debug:
        print(">> Anzahl '{'-Positionen:", len(brace_positions))

    for start in brace_positions:
        for end in range(start + 1, n):
            if s[end] == "}":
                candidate = s[start:end + 1]
                candidate_stripped = candidate.strip()
                # kleine Heuristik: mindestens 'tool' oder 'arguments' sollten drinstehen, sonst sparen wir uns json.loads
                if ("tool" not in candidate_stripped) and ("arguments" not in candidate_stripped):
                    continue

                if debug:
                    print(">>> Teste Kandidat (start={}, end={}):".format(start, end))
                    print(candidate_stripped)
                try:
                    data = json.loads(candidate_stripped)
                except json.JSONDecodeError:
                    if debug:
                        print(">>> Kandidat ist kein valides JSON.\n")
                    continue

                if not isinstance(data, dict):
                    if debug:
                        print(">>> Kandidat ist JSON, aber kein Dict.\n")
                    continue

                tool_name = data.get("tool")
                arguments = data.get("arguments")

                if isinstance(tool_name, str) and isinstance(arguments, dict):
                    if debug:
                        print(">>> Kandidat ist gültiger Tool-Call!")
                        print(f"    tool={tool_name}, arguments={arguments}\n")
                    return {"tool": tool_name, "arguments": arguments}

    if debug:
        print(">> Kein valider Tool-Call gefunden. Rückgabe = None.\n")

    return None

## Tool-Call parsen und inspizieren

### Funktion
- Ruft try_parse_tool_call(assistant_text) auf.
- Druckt den geparsten Tool-Call.

### Input
- assistant_text – Modellantwort aus dem „Koordinaten von Barcelona“-Run.

### Output
- parsed_call: {"tool": "...", "arguments": {...}} oder None.
- Konsolenausgabe des Ergebnisses.

In [39]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}}


## Ein MCP-Tool tatsächlich ausführen: call_mcp_tool_once

### Funktion
- Führt genau ein MCP-Tool aus und gibt ein „normales“ Python-Objekt zurück.
- Kapselt:
    - Aufbau der HTTP-Clientverbindung,
    - session.initialize(),
    - session.call_tool(tool_name, arguments).

### Input
- tool_name: str – z. B. "geocode".
- arguments: Dict[str, Any] – z. B. {"destination": "Barcelona"}.
- url: str = MCP_URL.

### Output
- Rückgabewert:
    - Primär: resp.structuredContent, falls vorhanden (oft schon ein Dict).
    - Sekundär: json.loads(content[0].text), wenn es Text-JSON gibt.
    - Fallback: ein einfaches Dict mit Metadaten (meta, isError) oder Text.
- Typischer Output (Dict):
    - {"lat": 41.38..., "lon": 2.17...}.

In [40]:
async def call_mcp_tool_once(
    tool_name: str,
    arguments: Dict[str, Any],
    url: str = MCP_URL,
) -> Any:
    """
    Führt genau EIN MCP-Tool aus und gibt ein "normales" Python-Objekt zurück
    (idealerweise ein dict), das direkt JSON-serialisierbar ist.
    """
    async with streamable_http_client(url) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            resp = await session.call_tool(tool_name, arguments)

            # Priorität 1: structuredContent (das ist bei dir schon ein dict)
            structured = getattr(resp, "structuredContent", None)
            if structured is not None:
                return structured

            # Priorität 2: content[0].text als JSON parsen, falls vorhanden
            content = getattr(resp, "content", None)
            if (
                content
                and isinstance(content, list)
                and hasattr(content[0], "text")
                and isinstance(content[0].text, str)
            ):
                txt = content[0].text
                try:
                    return json.loads(txt)
                except json.JSONDecodeError:
                    # Falls es kein JSON ist – dann geben wir einfach den Text zurück
                    return {"text": txt}

            # Fallback: zur Not alles in ein dict packen
            return {
                "meta": getattr(resp, "meta", None),
                "isError": getattr(resp, "isError", None),
            }


## Tool-Aufruf wirklich ausführen (Geocode Barcelona)

### Funktion
- Verwendet das geparste parsed_call, um das passende MCP-Tool auszuführen.
- Speichert den Tool-Output in tool_result.

### Input
- parsed_call["tool"], parsed_call["arguments"].
- MCP-Server muss laufen.

### Output
- tool_result – Ergebnisobjekt, z. B. Koordinaten für Barcelona.

### Konsole:
- tool_result: {...} zur Sichtkontrolle.

In [41]:
tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

[01/27/26 15:47:08] INFO     Created new          ]8;id=101543;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=858768;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             c7946aaa449849b4abe4                               
                             a82b58df89c5                                       
                    INFO     Processing request of type            ]8;id=875272;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=550914;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:41540 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41552 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:41562 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:41570 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:44856 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:44866 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'lat': 41.3825802, 'lon': 2.177073}


[01/27/26 15:47:09] INFO     Processing request of type            ]8;id=878040;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=456840;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py#713\713]8;;\
                             ListToolsRequest                                   
                    INFO     Terminating session:         ]8;id=533338;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=95443;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http.py#779\779]8;;\
                             c7946aaa449849b4abe4a82b58df                       
                             89c5                                               


## Tool-Historie initialisieren und füllen

### Funktion
- Legt eine Liste tool_history an.
- Hängt den eben ausgeführten Tool-Call samt Ergebnis daran an.
- Das ist die „Tool-Gedächtnisstruktur“ für spätere Prompts.

### Input
- tool_name, arguments, tool_result.

### Output
- tool_history: List[Dict[str, Any]]

### Konsole:
- print(tool_history).

In [42]:
# Tool-Verlauf speichern
tool_history = []

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

[{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}, 'result': {'lat': 41.3825802, 'lon': 2.177073}}]


## Koordinaten aus Tool-Historie nutzen, um die Nutzeranfrage final zu Beantworten

### Funktion
- Zeigt, wie das LLM nun auf Basis des bisherigen Tool-Ergebnisses antwortet.
- System: Tools sind erlaubt – aber allow_tools=False fügt einen Override-Systemprompt ein:
    - „Du darfst KEINE MCP-Tools aufrufen...“
- Da nun bereits alle notwendigen Informationen aus der MCP-Abfrage vorliegen

### Input
- tool_names_with_meta = await fetch_tools_with_metadata()
- user_input: wieder „Wie sind die genauen Geokoordinaten von Barcelona?“
- tool_history: enthält den geocode-Call + Result.
- allow_tools=False.

### Output
- assistant_text: LLM soll jetzt:
    - keine Tools mehr aufrufen,
    - die Koordinaten aus tool_history direkt als „Wissen“ verwenden
    - und im Fließtext antworten (z. B. „Barcelona liegt bei ca. Breitengrad X, Längengrad Y“).

### Konsole:
- Prompt + Antwort.

In [43]:
tool_names_with_meta = await fetch_tools_with_metadata()

user_input = "Ich möchte eine Reise nach Barcelona machen. Wie sind die genauen Geokoordinaten von Barcelona?"

history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/27/26 15:47:17] INFO     Created new          ]8;id=288358;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=828066;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             941c1f8a61a34a5d9adb                               
                             2e7ce9bdc24f                                       
                    INFO     Processing request of type            ]8;id=638270;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=523403;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:44872 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:44884 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:44900 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:44906 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:44914 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

## Erneute Wetteranfrage basierend auf dem zuvor erlangten Wissen der Koordinaten

### Funktion
- Jetzt kombinieren der Toolcalls von Hand:
    - Koordinaten von Barcelona sind bereits in tool_history.
    - User fragt erneut nach dem heutigen Wetter in Barcelona.
    - Da alle Argumente für get_weather nun vorhanden sind, darf das Modell ein Tool aufrufen.

### Input
- tool_names_with_meta = await fetch_tools_with_metadata()
- today = date.today()
- user_input: „Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona? Nutze die Geokoordinaten von vorhin…“
- tool_history: enthält geocode-Resultat aus vorherigem Call:
- allow_tools=True.

### Output
- assistant_text: sollte jetzt einen JSON-Tool-Call für get_weather produzieren.

### Konsole:
- Prompt + Modelloutput.

In [44]:
tool_names_with_meta = await fetch_tools_with_metadata()
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona? Nutze die Geokoordinaten von vorhin um das Wetter abzufragen."

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/27/26 15:51:53] INFO     Created new          ]8;id=643440;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=619856;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             5c7e9a81de9e4a4c8431                               
                             9b3cdb2669db                                       
                    INFO     Processing request of type            ]8;id=155462;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=309525;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:38032 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38048 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:38060 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38068 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:38072 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

Deine Aufgaben:
- Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
- Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
- Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

Nutzung von Kontext (RAG):
- Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
- Du 

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Output: ```json
{
  "tool": "get_weather",
  "arguments": {
    "lat": 41.3825802,
    "lon": 2.177073,
    "start_date": "2026-01-27",
    "end_date": "2026-01-27",
    "include_raw": false
  }
}
```




## Wetter-Tool ausführen und finale Antwort generieren

### Funktion
- Nächste manuelle Agentenrunde:
    - parsed_call = try_parse_tool_call(assistant_text) → Wetter-Tool-Call extrahieren.
    - tool_result = await call_mcp_tool_once(tool_name, arguments) → Wetterdaten holen.
    - tool_history.append(...) → Wetter-Resultat zur Historie hinzufügen.
     - Nochmals build_chat_prompt_with_rag_and_tools(..., allow_tools=False) mit denselben user_input + aktualisierter tool_history.
    - llama_chat(...) → jetzt Endantwort ohne neue Tools, aber auf Basis der gespeicherten Wetterdaten.

### Input
- assistant_text (mit Wetter-Tool-Call).
- MCP-Server, tool_history (mit geocode), user_input (Wetterfrage).
- allow_tools=False beim zweiten Prompt.

### Output
- parsed_call: Wetter-Tool-Call.
- tool_result: Wetterprofil (Open-Meteo Profil, z. B. Temperaturen, Niederschlag).
- Aktualisierte tool_history: enthält jetzt zwei Einträge (geocode, get_weather).
- assistant_text (zweiter LLM-Call):
- Modell erklärt das Wetter in Barcelona am heutigen Datum,
- ohne weitere Tools aufzurufen,
- und kann auf die strukturierten Wetterdaten zurückgreifen.

In [45]:
parsed_call = try_parse_tool_call(assistant_text)
print(parsed_call)

tool_name = parsed_call["tool"]
arguments = parsed_call["arguments"]

tool_result = await call_mcp_tool_once(tool_name, arguments)


print(f"tool_result: {tool_result}")

tool_history.append(
    {
        "tool": tool_name,
        "arguments": arguments,
        "result": tool_result,  # z.B. {'lat': 41.8933203, 'lon': 12.4829321}
    }
)

print(tool_history)

tool_names_with_meta = await fetch_tools_with_metadata()

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=system_prompt_for_tools_and_rag,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
    allow_tools=False,
)

print(f"Input: {prompt}\n\n")
print("Ende input\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

{'tool': 'get_weather', 'arguments': {'lat': 41.3825802, 'lon': 2.177073, 'start_date': '2026-01-27', 'end_date': '2026-01-27', 'include_raw': False}}
INFO:     127.0.0.1:39884 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39890 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:39902 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39908 - "POST /mcp HTTP/1.1" 200 OK


[01/27/26 15:52:18] INFO     Created new          ]8;id=78354;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=120393;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             e8f28c3f457e4c2dbf4c                               
                             8acda5a295c6                                       
                    INFO     Processing request of type            ]8;id=872219;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=284016;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

INFO:     127.0.0.1:39920 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:39928 - "DELETE /mcp HTTP/1.1" 200 OK
tool_result: {'start_date': '2026-01-27', 'end_date': '2026-01-27', 'days': [{'date': '2026-01-27', 'temp_min_c': 7.5, 'temp_max_c': 14.0, 'precipitation_mm': 5.4, 'wind_max_kmh': 17.8, 'weather_code': 61}], 'temp_min_c': None, 'temp_max_c': None, 'precip_total_mm': None, 'rainy_days': None, 'wind_max_kmh': None, 'raw': {}, 'source': 'forecast', 'is_estimate': False, 'reference_years': []}
[{'tool': 'geocode', 'arguments': {'destination': 'Barcelona'}, 'result': {'lat': 41.3825802, 'lon': 2.177073}}, {'tool': 'get_weather', 'arguments': {'lat': 41.3825802, 'lon': 2.177073, 'start_date': '2026-01-27', 'end_date': '2026-01-27', 'include_raw': False}, 'result': {'start_date': '2026-01-27', 'end_date': '2026-01-27', 'days': [{'date': '2026-01-27', 'temp_min_c': 7.5, 'temp_max_c': 14.0, 'precipitation_mm': 5.4, 'wind_max_kmh': 17.8, 'weather_code': 61}], 'temp_min_c': None, 'tem

# Teil 4: Voll agentisches MCP – der Reiseagent denkt und handelt selbst

In Teil 4 machen wir aus dem „Half-Agent“ aus Teil 3 jetzt einen echten, voll agentischen Workflow:
- Das LLM plant eine Tool-Kette (inkl. Parallelität).
- Es referenziert frühere Tool-Outputs über $ref.
- Ein Steuerloop (agentic_run_to_final_answer) führt Schritt für Schritt alle geplanten Tools aus, aktualisiert die Historie und lässt das Modell bei Bedarf re-planen, bis eine finale Antwort entsteht.

Konzeptionell lehnen wir uns an Tool-/Plan-basierte Agenten wie ReAct und ähnliche Frameworks an, die iterativ zwischen „Denken“, „Handeln (Tool)“ und „Beobachten“ wechseln.

## Agenten-Systemprompt mit Tool-Chains: build_agent_system_prompt

### Funktion
- Baut einen umfangreichen System-Prompt für einen agentischen Reiseplaner, der:
    - RAG-Kontext nutzen darf,
    - MCP-Tools verwenden darf,
    - Tool-Ketten planen soll (ReAct-Style Plan),
    - Ausgaben immer als JSON-Plan mit steps + final zurückgibt,
    - $ref-Mechanismus für Abhängigkeiten zwischen Tool-Outputs und Folgetools nutzt,
    - User-Informationen via user.*-Referenzen einbindet.

### Input
- tools: List[Any] – Toolobjekte vom MCP-Server, z. B. geocode, get_weather, get_spots, rank_spots.
- Intern wird format_tools_for_prompt(tools) aufgerufen, um einen detaillierten Toolkatalog (inkl. JSON-Schemas) in den Prompt einzubauen.

### Output
- agent_system_prompt: str – System-Prompt, der u. a. festlegt:
    - Ausgabeformat des Modells.
    - Semantik von steps:
        - alle tools in einem steps[i] sind parallel ausführbar,
        - spätere Schritte können via $ref auf Tool-Outputs früherer Schritte zugreifen.
    - $ref-Konventionen:
        - { "$ref": "geo_barcelona.lat" } → greift auf result["lat"] eines Tool-Calls mit ID geo_barcelona zu.
        - { "$ref": "user.raw" } oder user.destination etc. für Teile der Nutzereingabe.
    - Einschränkungen:
        - Nur gültiges JSON oder final Text ausgeben,
        - keine mehrfachen Keys,
        - arguments ist immer ein Objekt mit benannten Feldern, keine nackten $ref.
    - Verhalten bei Out-of-Scope:
        - steps: [], final: "Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht."

In [46]:
from typing import Any, List

def build_agent_system_prompt(tools: List[Any]) -> str:
    tool_catalog = format_tools_for_prompt(tools)

    system_prompt = """
    Du bist ein persönlicher Reiseplaner.

    Deine Aufgaben:
    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

    Nutzung von Kontext (RAG):
    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
    - Du darfst alle darin enthaltenen Fakten verwenden.
    - Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

    Allgemeines Arbeitsprinzip (ReAct mit Planung und Tool-Chains):
    - Denke zuerst still über einen mehrschrittigen Plan nach, wie du die Anfrage des Nutzers beantworten kannst.
    - Plane dabei explizit:
      - welche Tools du brauchst,
      - welche Tools voneinander abhängen (sequentiell) und
      - welche Tools unabhängig voneinander parallel ausgeführt werden können.
    - Gruppiere parallel ausführbare Tools in demselben Schritt.
    - Tools, die Ausgaben anderer Tools benötigen, müssen in einem späteren Schritt stehen und ihre Argumente über Referenzen auf frühere Tool-IDs erhalten.
    - Nach der Ausführung eines Schritts darf der Plan in einem späteren Turn angepasst oder erweitert werden, falls Zwischenergebnisse neue Informationen liefern.

    Ausgabeformat DEINER Antwort:
    - Du antwortest IMMER mit genau einem JSON-Objekt (kein Fließtext außerhalb des JSON, kein Markdown, keine ```-Codeblöcke).
    - Das JSON hat IMMER die Form:

    ```json
    {
      "steps": [
        {
          "description": "<natürliche Beschreibung des Schritts>",
          "tools": [
            {
              "id": "<eindeutige-id>",
              "name": "<tool-name>",
              "arguments": {
                /* Argumente, ggf. mit $ref-Platzhaltern */
              }
            }
          ]
        }
        /* weitere Schritte */
      ],
      "final": null oder "<natürliche Antwort für den Nutzer>"
    }
    - Die Semantik von "steps":
      - Jeder Eintrag in "steps" beschreibt einen logischen Schritt im Plan.
      - Alle Tools innerhalb desselben "steps[i].tools" sollen parallel ausführbar sein (keine gegenseitigen Abhängigkeiten).
      - Tools, die von Ergebnissen früherer Tools abhängen, müssen in einem späteren Schritt (höherer Index) stehen.
    - Wenn du noch Tools brauchst:
      - Fülle "steps" mit dem geplanten Tool-Plan.
      - Setze "final": null.
    - Wenn du keine Tools mehr brauchst und alle nötigen Informationen hast:
      - Kannst du "steps": [] setzen (oder nur noch eine beschreibende Abschlussaktion ohne Tools) und
      - "final" mit der fertigen, gut lesbaren Antwort für den Nutzer füllen.
    - Es ist erlaubt, in derselben Antwort sowohl weitere Schritte mit Tools als auch schon eine vorbereitete finale Antwort vorzusehen, aber der Normalfall ist:
      - Zwischenturn: "final": null, Fokus auf Tool-Plan.
      - Letzter Turn: "steps": [] und "final": "<Antwort…>".
    - WICHTIG: Gültiges JSON
      - Jeder JSON-Output MUSS syntaktisch gültig sein.
      - In einem JSON-Objekt darf jeder Schlüssel nur EINMAL vorkommen.
        Beispiel für UNGÜLTIG (verboten):
        {
        "$ref": "geo_roma.lat",
        "$ref": "geo_roma.lon"
        }
      - Stattdessen musst du für jedes Feld einen eigenen Key verwenden und dort ggf. einen $ref-Wert eintragen, z.B.:
        {
        "lat": { "$ref": "geo_roma.lat" },
        "lon": { "$ref": "geo_roma.lon" }
        }
      - Schreibe KEIN "arguments": { "$ref": "..." } ohne weitere Felder. Immer: "arguments": { "<argument-name>": <Wert oder $ref-Objekt>, ... }.
    - Spezifikation der Tool-Objekte:
      - Jedes Tool-Objekt in "tools" hat:
        - "id": eine eindeutige Bezeichner-String innerhalb dieses Plans (z.B. "geo_berlin").
        - "name": genau den Namen eines der verfügbaren MCP-Tools (siehe unten "Tools:").
        - "arguments": ein JSON-Objekt, das exakt dem Eingabe-Schema des jeweiligen Tools entspricht.
      - "arguments" MUSS ein Objekt mit den im Input-Schema definierten Feldnamen sein.
        Beispiel (vereinfacht) für das Tool "get_weather", dessen Schema u.a. "lat", "lon", "start_date", "end_date" vorsieht:
        "arguments": {
        "lat": { "$ref": "geo_roma.lat" },
        "lon": { "$ref": "geo_roma.lon" },
        "start_date": "2026-01-23",
        "end_date": "2026-01-23",
        "include_raw": false
        }
      - Du darfst KEIN "arguments": { "$ref": "..." } erzeugen. Jeder Parameter des Tools muss entweder:
        - ein literaler Wert sein (z.B. "2026-01-23", 5, true) ODER
        - ein Objekt der Form { "$ref": "<quelle.pfad>" }.
      - Referenzen für Argumente ($ref):
        - KEIN Argument darf aus der Luft erfunden werden. Wenn ein Argument aus einem Tool-Output oder aus dem User-Input stammt, markiere das explizit mit einem $ref-Objekt.
      - Grundform von $ref:
        - Ein $ref-Wert hat IMMER die Form:
          - { "$ref": "<quelle>" }
        - Der String <quelle> hat die Form "<tool-id>" oder "<tool-id>.<pfad>" oder "user.<pfad>".
        - Ergebnisse früherer Tools:
          Die Ergebnisse früherer Tools werden dir in der History als Messages der Form gezeigt:
          {
          "role": "tool",
          "tool_id": "<tool-id>",
          "name": "...",
          "arguments": { ... },
          "result": { ... }
          }
        - Für $ref verwendest du IMMER die "tool_id" als Wurzel.
        - Der Pfad <pfad> bezieht sich auf die Felder innerhalb von "result".
          Beispiel-History:
          {
          "role": "tool",
          "tool_id": "geo_roma",
          "result": {
          "lat": 41.8933,
          "lon": 12.4829
          }
          }

          → Zulässige $ref:
          { "$ref": "geo_roma" } → das komplette result-Objekt
          { "$ref": "geo_roma.lat" } → 41.8933
          { "$ref": "geo_roma.lon" } → 12.4829
        - WICHTIG:
          - Du darfst NICHT auf "result" selbst referenzieren, also KEIN: { "$ref": "geo_roma.result" }
          - Du gehst IMMER so vor, als ob der Inhalt von "result" die Wurzelstruktur für $ref ist.
          - Pfade wie "geo_roma.arguments" oder "geo_roma.result.lat" sind VERBOTEN.
        - Beispiel für korrektes Verwenden von Geo-Koordinaten in einem Folge-Tool "get_weather":
          "steps": [
          {
          "description": "Geokoordinaten für Rom bestimmen",
          "tools": [
          {
          "id": "geo_roma",
          "name": "geocode",
          "arguments": {
          "destination": "Rom"
          }
          }
          ]
          },
          {
          "description": "Tageswetter für Rom am gewünschten Datum abrufen",
          "tools": [
          {
          "id": "wetter_roma",
          "name": "get_weather",
          "arguments": {
          "lat": { "$ref": "geo_roma.lat" },
          "lon": { "$ref": "geo_roma.lon" },
          "start_date": "2026-01-23",
          "end_date": "2026-01-23",
          "include_raw": false
          }
          }
          ]
          }
          ]
        - Nutzer-Eingabe:
          - Wenn ein Argument direkt aus der aktuellen Nutzernachricht kommen soll (z.B. Stadtname, Datum, Budget), verwende eine Referenz mit Präfix "user.".
          - Beispiele:
              - { "$ref": "user.origin" } → Herkunftsort aus der Nutzernachricht.
              - { "$ref": "user.destination" } → Zielort aus der Nutzernachricht.
              - { "$ref": "user.raw" } → komplette Roh-Eingabe des Nutzers.
          - Du definierst diese Pfade semantisch, damit für das Backend klar ist, welcher Teil des User-Inputs gemeint ist.
        - Zusammenfassung für $ref:
          - $ref steht IMMER als Wert eines Feldes in "arguments".
          - Du verwendest KEIN "arguments": { "$ref": ... } ohne weitere Felder.
          - Du referenzierst NIE "result", "arguments" oder ähnliche Meta-Felder, sondern arbeitest so, als ob die "result"-Struktur selbst die Basis für den Pfad ist.
        - Planung von Tool-Chains:
          - Du DARFST und SOLLST Tool-Ketten planen (z.B. "zuerst Geocoding, dann Routenberechnung, dann Bewertung").
          - Überlege explizit, welche Tools Vorbedingungen anderer Tools erfüllen (z.B. Koordinaten, IDs, Zeiträume).
          - Baue den Plan so, dass:
            - Schritt 0: Tools, die direkt aus dem User-Input und ggf. Kontext ihre Argumente beziehen.
            - Schritt 1: Tools, deren Argumente via $ref aus Ergebnissen von Schritt 0 kommen.
            - Schritt 2: Tools, die auf Schritt 1 aufbauen, usw.
          - Tools, die keine gemeinsamen Abhängigkeiten haben, dürfen im selben Schritt stehen, damit sie parallel ausgeführt werden können.
          - Du darfst mehrere Schritte vorausplanen. Das Backend kann nach Ausführung eines oder mehrerer Schritte den Plan durch einen erneuten Aufruf des Modells aktualisieren lassen.
          - Wenn du einen bestehenden Plan aktualisierst erhalten die bereits ausgeführten Schritte die selben Bezeichner wie zuvor. Toolnamen ausgeführter Tools dürfen Rückwirkend also nie verändert werden.
        - Nutzung von MCP-Tools:
          - Nutze Tools, wenn du:
            - Informationen brauchst, die nicht sicher in deinem Weltwissen oder im bereitgestellten Kontext stehen, oder
            - exakte Daten (z.B. Koordinaten, Preise, Zeiten, Verfügbarkeiten) benötigst.
          - Nutze Tools nur dann, wenn du ihre Input-Schemata sinnvoll und vollständig (ggf. mit $ref) befüllen kannst.
          - Verwende für Tool-Argumente ausschließlich:
            - explizite Informationen aus der aktuellen Nutzernachricht (per $ref "user.…"),
            - frühere Tool-Ergebnisse (per $ref auf Tool-IDs),
            - oder wohldefinierte konstante/literale Werte.
          - Du DARFST Tool-Argumente NICHT aus deinem allgemeinen Weltwissen raten.
          - Wenn die Anfrage ohne Tools gut beantwortbar ist (reine Beratung, Einschätzung, Inspiration), dann erstelle einen JSON-Output mit:
            - "steps": [] (oder nur erklärende Schritte ohne Tools) und
            - "final": "<deine natürliche Antwort>".
        - Umgang mit Wissen und Unsicherheit:
          - Erfinde keine Fakten und spekuliere nicht.
          - Wenn eine Information weder in deinem Weltwissen noch im bereitgestellten Kontext vorkommt und auch nicht per MCP-Tool-Call erlangt werden kann, erkläre dies in der "final"-Antwort.
          - Wenn dir wirklich Informationen fehlen, formuliere in "final" eine ehrliche, erklärende Antwort und beschreibe, was du stattdessen sicher sagen kannst.
        - Einschränkungen (Ablehnungsfälle im Reise-Kontext):
          - Lehne Anfragen ab, die eindeutig NICHT zum Reise-/Tourismuskontext gehören.
          - In diesem Fall gibst du ebenfalls ein JSON-Objekt zurück, z.B.:
            {
            "steps": [],
            "final": "Diese Frage kann ich nicht beantworten, da sie nicht dem Kontext des Assistenten entspricht."
            }
        - Tools (verfügbare MCP-Tools):
    """
    return system_prompt + tool_catalog

## Agent-Systemprompt bauen

### Funktion
- Ruft build_agent_system_prompt(tool_names_with_meta) auf, um den finalen Prompt-String zu erzeugen.

### Input
- tool_names_with_meta – Toolmetadaten, wie zuvor per fetch_tools_with_metadata() geladen.

### Output
- agent_system_prompt: str – wird später in allen agentischen Calls verwendet.

### Konsole
- Ausgabe von agent_system_prompt

In [47]:
agent_system_prompt = build_agent_system_prompt(tool_names_with_meta)
print(agent_system_prompt)


    Du bist ein persönlicher Reiseplaner.

    Deine Aufgaben:
    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

    Nutzung von Kontext (RAG):
    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.
    - Du darfst alle darin enthaltenen Fakten verwenden.
    - Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.

    Allgemeines Arbeitsprinzip (ReAct mit Planung und Tool-Chains):
    - Denke zuerst still über einen mehrschrittigen Plan nach, wie du die Anfrage des Nutzers beantworten kannst.
    - Plane dabei 

## Erster agentischer Call: Wetter heute in Barcelona (Wir wollen uns erst mal nur den Plan-Output ansehen)

### Funktion
- Testet das neue agentische Setup:
    - User fragt: „Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona?“
    - LLM sollte keine direkte Antwort, sondern einen JSON-Plan ähnlich diesem liefern:
        - Schritt 1: geocode("Barcelona")
        - Schritt 2: get_weather(lat, lon, start_date=today, end_date=today, include_raw=false) mit $ref auf Schritt 1.

### Input
- tool_names_with_meta = await fetch_tools_with_metadata()
- today = date.today()
- user_input: Frage mit Datum.
- tool_history = None
- Promptbau über build_chat_prompt_with_rag_and_tools(...):
- system_prompt=agent_system_prompt
- history=leere History
- tool_history=None – noch keine Tools gelaufen.

### Output
- prompt: str – Chat-Prompt inkl. System-Agentenprompt.
- assistant_text: Modelloutput, der idealerweise ein gültiges JSON-Objekt im beschriebenen Planformat ist.
- Enthält steps (mit Toolkette) und oft final: null im ersten Schuss.

### Konsole:
- Input-Prompt und Output (Plan) zum Debuggen.

In [48]:
tool_names_with_meta = await fetch_tools_with_metadata()
from datetime import date

# Returns the current local date
today = date.today()

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona?"
tool_history = None
history = []

prompt = build_chat_prompt_with_rag_and_tools(
    system_prompt=agent_system_prompt,
    user_prompt=user_input,
    history=history,
    tool_history=tool_history,
)

print(f"Input: {prompt}\n\n")
assistant_text = llama_chat(prompt, max_new_tokens=512)
print(f"Output: {assistant_text}\n\n")

[01/27/26 15:53:09] INFO     Created new          ]8;id=582389;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=932033;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             6587f04505ab4671bb54                               
                             ea40a5e399d4                                       
                    INFO     Processing request of type            ]8;id=372393;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=553540;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

INFO:     127.0.0.1:59774 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59780 - "POST /mcp HTTP/1.1" 202 Accepted
INFO:     127.0.0.1:59786 - "GET /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59800 - "POST /mcp HTTP/1.1" 200 OK
INFO:     127.0.0.1:59804 - "DELETE /mcp HTTP/1.1" 200 OK
Input: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Du bist ein persönlicher Reiseplaner.

    Deine Aufgaben:
    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.
    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).
    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.

    Nutzung von Kontext (RAG):
    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für d

# Im Anschluss werden wir die bearbeitung dieses Plans automatisieren

## Agentischen Plan robust parsen: agentic_parse_model_plan

### Funktion
- Extrahiert das JSON-Objekt mit steps und final aus dem Modelloutput, selbst wenn das Modell charme-typisch etwas „drumrum“ schreibt.
- Robust gegen:
    - Vor-/Nachtext,
    - kleine Formatabweichungen, solange irgendwo ein gültiges JSON-Objekt {...} enthalten ist.

### Input
- output_str: str – der rohe String von llama_chat.
    - Funktion loggt Debug-Ausgaben:
        - Raw output
        - „trimmed JSON candidate“

### Output
- plan: Dict[str, Any] – geparstes Planobjekt mit:
    - Pflichtfeldern:
        - "steps": List
        - "final": beliebiger Wert (auch null → None)
    - Fehlerszenarien:
        - Kein { gefunden → ValueError("No JSON object found...")
        - JSON-Parsefehler → ValueError(...)
        - "steps" fehlt oder ist kein List → ValueError.

In [49]:
import json
from typing import Any, Dict

def agentic_parse_model_plan(output_str: str) -> Dict[str, Any]:
    print("[DEBUG] raw output:", repr(output_str))

    # Leer- oder Nonsens-Output früh abfangen
    if not output_str or not output_str.strip():
        raise ValueError("Model output is empty, cannot parse JSON plan.")

    # Die erste öffnende Klammer suchen
    start = output_str.find("{")
    if start == -1:
        raise ValueError(f"No JSON object found in model output: {output_str!r}")

    s = output_str

    # Jetzt von 'start' an die passende schließende Klammer suchen
    depth = 0
    in_string = False
    escape = False
    end: Optional[int] = None

    for i, ch in enumerate(s[start:], start=start):
        if in_string:
            # Innerhalb eines Strings
            if escape:
                # nächstes Zeichen ist escaped, einfach überspringen
                escape = False
            elif ch == "\\":
                escape = True
            elif ch == '"':
                in_string = False
        else:
            # Außerhalb von Strings
            if ch == '"':
                in_string = True
            elif ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0:
                    end = i
                    break

    if end is None:
        # Fallback: alter Modus (kannst du auch wegwerfen, wenn du möchtest)
        trimmed = s[start:]
    else:
        trimmed = s[start:end + 1]

    print("[DEBUG] trimmed JSON candidate:", trimmed)

    try:
        plan = json.loads(trimmed)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Failed to parse JSON plan from model output. "
            f"Error: {e}. Trimmed candidate: {trimmed!r}"
        ) from e

    if "steps" not in plan or "final" not in plan:
        raise ValueError("Model output must contain 'steps' and 'final'.")

    if not isinstance(plan["steps"], list):
        raise ValueError("'steps' must be a list.")

    return plan

## $ref-Resolver – Einzelwert: agentic_resolve_argument

### Funktion
- Löst ein einzelnes Argument auf, das entweder:
    - ein literaler Wert ist (String, Zahl, Bool),
    - ein Dict mit {"$ref": "..."},
    - oder ein String im Format "$ref:...".
- Unterstützt Referenzen auf:
    - Tool-Resultate ("geo_barcelona.lat"),
    - User-Kontext ("user.raw" oder z. B. "user.destination").

### Input
- value: Any – roher Argumentwert aus dem Plan.
- tool_results: Dict[str, Any] – Mapping tool_id -> result (ohne Meta).
- user_ctx: Dict[str, Any] – z. B. {"raw": user_input}.

### Output
- Aufgelöster Wert:
    - Wenn kein $ref: Wert unverändert.
    - Wenn $ref:
        - Traversiert Pfadsegmente (split(".")) über Dictionaries/Listen.
        - Gibt konkreten Wert zurück (z. B. 41.38... für Lat).
- Fehler:
    - Unbekannte tool_id oder ungültiger Pfad → KeyError.

In [50]:
from typing import Any, Dict, Optional

def agentic_resolve_argument(
    value: Any,
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> Any:
    """
    Löst einzelne Argumentwerte auf. Unterstützt:
    - {"$ref": "geo_roma.lat"}
    - "$ref:geo_roma.lat"
    - "$ref:user.raw"
    """
    ref_str: Optional[str] = None

    # Fall 1: Objekt mit "$ref"
    if isinstance(value, dict) and "$ref" in value and isinstance(value["$ref"], str):
        ref_str = value["$ref"]

    # Fall 2: String mit Präfix "$ref:"
    elif isinstance(value, str) and value.startswith("$ref:"):
        ref_str = value[len("$ref:") :]

    # Kein $ref → Wert unverändert zurückgeben
    if ref_str is None:
        return value

    # Jetzt ref_str auswerten, z.B. "geo_roma.lat" oder "user.raw"
    parts = ref_str.split(".")
    root = parts[0]
    path = parts[1:]

    # Quelle: User-Kontext
    if root == "user":
        current: Any = user_ctx
    else:
        # Quelle: Tool-Resultate
        if root not in tool_results:
            raise KeyError(f"Unknown tool id in $ref: {root}")
        current = tool_results[root]

    # Debug: einmal zeigen, was wir da wirklich haben
    print(f"[DEBUG] Resolving $ref '{ref_str}': root='{root}', initial_type={type(current)}, initial_value={current}")

    for p in path:
        # Numerische Indizes unterstützen (z.B. itineraries.0)
        if isinstance(current, list) and p.isdigit():
            idx = int(p)
            current = current[idx]
        elif isinstance(current, dict) and p in current:
            current = current[p]
        else:
            # Noch mehr Debug, bevor wir crashen
            raise KeyError(
                f"Invalid path component '{p}' in $ref '{ref_str}' "
                f"(current_type={type(current)}, current_value={current})"
            )

    return current

## $ref-Resolver – rekursiv über Argumente: agentic_resolve_tool_arguments

### Funktion
- Wendet agentic_resolve_argument auf komplette arguments-Strukturen an (Dicts/Listen).
- Unterstützt:
    - verschachtelte Strukturen,
    - Listen von Werten,
    - Spezialfall: {"$ref": "..."}
        - wird komplett durch das referenzierte Objekt ersetzt.

### Input
- arguments: beliebige JSON-ähnliche Struktur aus dem Plan (dict, list, primitive).
- tool_results, user_ctx – wie oben.

### Output
- Neue Struktur mit allen $ref durch konkrete Werte ersetzt
    - z. B.:

    ```python
    {
      "lat": {"$ref": "geo_barcelona.lat"},
      "lon": {"$ref": "geo_barcelona.lon"},
      "start_date": "2026-01-23"
    }
    ```

    - → nach Auflösung:

    ```python
    {
      "lat": 41.38,
      "lon": 2.17,
      "start_date": "2026-01-23"
    }
    ```

In [51]:
def agentic_resolve_tool_arguments(arguments, tool_results, user_ctx):
    if isinstance(arguments, dict):
        # Spezialfall: dict besteht NUR aus "$ref" → ersetze das ganze Dict durch das referenzierte Objekt
        if set(arguments.keys()) == {"$ref"} and isinstance(arguments["$ref"], str):
            return agentic_resolve_argument(arguments, tool_results, user_ctx)

        return {
            k: agentic_resolve_tool_arguments(v, tool_results, user_ctx)
            for k, v in arguments.items()
        }
    elif isinstance(arguments, list):
        return [
            agentic_resolve_tool_arguments(v, tool_results, user_ctx)
            for v in arguments
        ]
    else:
        return agentic_resolve_argument(arguments, tool_results, user_ctx)

## Einen Step ausführen: agentic_execute_step

### Funktion
- Führt alle Tools in einem Plan-Step aus:
    - löst zuerst alle $ref in arguments auf,
    - ruft dann für jedes Tool das MCP-Backend auf,
    - prüft auf Fehler,
    - speichert Ergebnisse in tool_results,
    - erzeugt History-Einträge im Format:
    ```python
    {
      "role": "tool",
      "tool_id": "...",
      "name": "...",
      "arguments": {...},
      "result": {...},
    }
    ```

### Input
- step: Dict[str, Any] – ein Eintrag aus plan["steps"], z. B.:
```python
{
  "description": "...",
  "tools": [
    {"id": "geo_barcelona", "name": "geocode", "arguments": {...}},
    ...
  ]
}
```
- call_tool_fn: Callable, das ein Tool wirklich ausführt (agentic_call_mcp_tool).
- tool_results: Dict[str, Any] – wird in-place erweitert.
- user_ctx: Dict[str, Any] – für user.*-Referenzen.

### Output
- history_entries: List[Dict[str, Any]] – Tool-History-Objekte, die später wieder in den Prompt eingebaut werden.

### Side-Effects:
- Debug-Prints zu aufgelösten Argumenten und Ergebnissen.
- tool_results[tool_id] = res für alle Tools des Steps.
- Fehlerthrow, wenn normalized_result.get("isError") true ist.

In [52]:
from typing import Callable, Awaitable

async def agentic_execute_step(
    step: Dict[str, Any],
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    tool_results: Dict[str, Any],
    user_ctx: Dict[str, Any],
) -> List[Dict[str, Any]]:
    """
    Führt alle Tools in einem Step aus.
    - step: {"description": "...", "tools": [ {id, name, arguments}, ... ]}
    - call_tool_fn: deine MCP-Invoker-Funktion (z.B. call_mcp_tool)
    - tool_results: wird in-place mit neuen Ergebnissen gefüllt
    - user_ctx: z.B. {"raw": user_input}

    Rückgabe: Liste von History-Objekten für den Prompt (z.B. Tool-Logs).
    """
    history_entries: List[Dict[str, Any]] = []

    for tool in step.get("tools", []):
        tool_id = tool["id"]
        tool_name = tool["name"]
        raw_args = tool.get("arguments", {})

        # $ref auflösen
        resolved_args = agentic_resolve_tool_arguments(raw_args, tool_results, user_ctx)

        # Tool aufrufen
        print(f"[{tool_id}] {tool_name} resolved arguments: {resolved_args}")
        result = await call_tool_fn(tool_name, resolved_args)
        print(f"toolcallresult: {result}")

        tool_id = tool.get("id")

        # MCP CallToolResult → direkt structuredContent speichern
        if hasattr(result, "structuredContent") and result.structuredContent is not None:
            normalized_result = result.structuredContent
        # Falls du irgendwo schon dicts baust, die structuredContent enthalten
        elif isinstance(result, dict) and "structuredContent" in result and result["structuredContent"] is not None:
            normalized_result = result["structuredContent"]
        else:
            normalized_result = result

        print(f"[DEBUG] Stored tool_result[{tool_id}] = {type(normalized_result)} → {normalized_result}")

        # History-Eintrag für Tools (kannst du an dein Format anpassen)

        if normalized_result.get("isError"):
            raise Exception(f"Tool failed with error: {normalized_result.get('isError')}")

        if isinstance(normalized_result, dict) and "data" in normalized_result:
            res = normalized_result["data"]
        else:
            res = normalized_result

        # Ergebnis speichern für for Schleife bei parallelen Tools
        tool_results[tool_id] = res

        history_entries.append({
            "role": "tool",
            "tool_id": tool_id,
            "name": tool_name,
            "arguments": resolved_args,
            "result": res,
        })

    return history_entries

## Nächsten Step im Plan finden: agentic_find_next_step

### Funktion
- Wählt den nächsten noch nicht ausgeführten Step anhand des Plan-Index.
- Simple Strategie: der kleinste Index, der noch nicht in executed_step_indices steht.

### Input
- plan: Dict[str, Any] – der aktuelle Plan.
- executed_step_indices: Set[int] – Menge bereits ausgeführter Step-Indizes.

### Output
- int – Index des nächsten Steps, oder
- None, wenn alle Steps ausgeführt wurden.

In [53]:
from typing import Set

def agentic_find_next_step(
    plan: Dict[str, Any],
    executed_step_indices: Set[int],
) -> Optional[int]:
    """
    Wählt den nächsten noch nicht ausgeführten Step.
    V1: simpel – der kleinste Index, der noch nicht in executed_step_indices ist.
    """
    for idx, _ in enumerate(plan.get("steps", [])):
        if idx not in executed_step_indices:
            return idx
    return None

## Prompt-Builder für den Agenten: agentic_build_chat_prompt_with_rag_and_tools

### Funktion
- Baut einen Llama-konformen Chat-Prompt im rohen Template-Format, statt tokenizer.apply_chat_template zu nutzen.
- Integriert:
    - Systemprompt (Agentenlogik + Toolbeschreibungen),
    - optionalen RAG-Kontext,
    - bisherige Chat-History (role / content),
    - Tool-History (als role="tool" mit JSON-Content),
    - aktuelle User-Nachricht.

### Input
- system_prompt: str – hier: agent_system_prompt.
- user_prompt: str – aktuelle Userfrage.
- history: List[Dict[str, Any]] – z. B. frühere User/Assistant-Messages.
- tool_history: Optional[List[Dict[str, Any]]] – bisherige Tool-Calls.
- retrieved_context: Optional[str] – optionaler RAG-Kontext (hier noch nicht aktiv genutzt, aber vorbereitet).

### Output
- str: Promptstring mit Llama-Spezialtokens:
- <|begin_of_text|>, <|start_header_id|>system<|end_header_id|>, <|eot_id|>, etc.
- Wird direkt an llama_chat(prompt, ...) übergeben.

In [54]:
def agentic_build_chat_prompt_with_rag_and_tools(
    system_prompt: str,
    user_prompt: str,
    history: List[Dict[str, Any]],
    tool_history: Optional[List[Dict[str, Any]]] = None,
    retrieved_context: Optional[str] = None,
) -> str:
    """
    Baut den vollständigen Prompt für das LLM, inkl.:
    - system_prompt
    - optionaler Kontext (RAG)
    - bisherige Chat-History
    - bisherige Tool-History
    - aktuelle User-Nachricht
    """
    parts = []
    parts.append("<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n")
    parts.append(system_prompt)
    parts.append("<|eot_id|>")

    if retrieved_context:
        parts.append("<|start_header_id|>system<|end_header_id|>\n\n")
        parts.append(f"Kontext (aus Retrieval):\n{retrieved_context}\n")
        parts.append("<|eot_id|>")

    # alte history anhängen
    if history:
        for msg in history:
            parts.append(f"<|start_header_id|>{msg['role']}<|end_header_id|>\n\n")
            parts.append(msg["content"])
            parts.append("<|eot_id|>")

    # tool_history (falls du sie als eigene „role“ einfügst)
    if tool_history:
        for th in tool_history:
            parts.append(f"<|start_header_id|>tool<|end_header_id|>\n\n")
            parts.append(json.dumps(th, ensure_ascii=False))
            parts.append("<|eot_id|>")

    # Aktuelle Userfrage
    parts.append("<|start_header_id|>user<|end_header_id|>\n\n")
    parts.append(user_prompt)
    parts.append("<|eot_id|>")

    return "".join(parts)

## Hauptschleife des Agenten: agentic_run_to_final_answer

### Funktion
- Orchestriert die gesamte agentische Interaktion:
    1. Baut Prompt (inkl. Tool-History).
    2. Holt Plan vom LLM.
    3. Wählt nächsten Step, führt Tools aus, ergänzt Tool-History.
    4. Erlaubt Re-Planning in der nächsten Iteration.
    5. Beendet, wenn final gesetzt und keine Steps mehr offen sind.

### Input
- user_input: str – ursprüngliche Nutzerfrage.
- system_prompt: str – agent_system_prompt.
- history: List[Dict[str, Any]] – z. B. frühere Interaktion (Name „Max Mustermann“).
- tool_history: Optional[List[Dict[str, Any]]] – initial meist None.
- call_tool_fn: z. B. agentic_call_mcp_tool (führt MCP-Tools aus).
- build_prompt_fn: Standard agentic_build_chat_prompt_with_rag_and_tools.
- max_iterations: int = 8 – Sicherheitsgrenze gegen Endlosschleifen.

### Output
- str: finale Antwort an den Nutzer (plan["final"]), z. B. eine Beschreibung des Wetters oder der besten Jogging-Spots.

### Side-Effects:
- Debug-Ausgaben für jede Iteration:
    - Prompt,
    - Modelloutput,
    - Plan,
    - gewählten Step, etc.
    - Tool-History wächst über die Iterationen.

In [55]:
async def agentic_run_to_final_answer(
    user_input: str,
    system_prompt: str,
    history: List[Dict[str, Any]],
    tool_history: Optional[List[Dict[str, Any]]],
    call_tool_fn: Callable[[str, Dict[str, Any]], Awaitable[Any]],
    build_prompt_fn: Callable[..., str] = agentic_build_chat_prompt_with_rag_and_tools,
    max_iterations: int = 8,
) -> str:
    """
    Hauptschleife:
    - plant mit dem LLM,
    - führt Step für Step Tools aus,
    - replanned,
    - liefert am Ende plan['final'] zurück.
    """
    if tool_history is None:
        tool_history = []

    tool_results: Dict[str, Any] = {}

    executed_step_indices: Set[int] = set()

    # User-Kontext für $ref: "user.*"
    user_ctx = {
        "raw": user_input,
        # hier könntest du später origin/destination/etc. extrahieren
    }

    for iteration in range(max_iterations):
        # 1) Prompt bauen + Plan vom Modell holen
        prompt = build_prompt_fn(
            system_prompt=system_prompt,
            user_prompt=user_input,
            history=history,
            tool_history=tool_history,
        )

        print(f"iteration: {iteration}, prompt: {prompt!r}")
        assistant_text = llama_chat(prompt, max_new_tokens=1024)
        print(f"iteration: {iteration}, assistant_text: {assistant_text!r}")

        # 2) Plan parsen
        plan = agentic_parse_model_plan(assistant_text)
        print(f"iteration: {iteration}, plan: {plan!r}")

        # 3) Wenn final schon vorhanden und keine offenen Steps → fertig
        next_idx = agentic_find_next_step(plan, executed_step_indices)
        print(f"iteration: {iteration}, next_idx: {next_idx!r}")

        if plan.get("final") is not None and next_idx is None:
            # finale Antwort
            final = plan["final"]
            print(f"iteration: {iteration}, final: {final}")
            return plan["final"]

        # 4) Falls es einen nächsten Step gibt → ausführen
        if next_idx is not None:
            step = plan["steps"][next_idx]
            step_history_entries = await agentic_execute_step(
                step=step,
                call_tool_fn=call_tool_fn,
                tool_results=tool_results,
                user_ctx=user_ctx,
            )
            executed_step_indices.add(next_idx)

            # Tool-Historie erweitern (für nächsten Prompt)
            tool_history.extend(step_history_entries)

            # Nach jedem Step neue Iteration starten → replanning möglich
            continue

        # 5) Kein Step mehr, aber final ist noch None → Modell „nötigen“, final zu setzen
        if plan.get("final") is None and next_idx is None:
            # Einfach eine weitere Runde, diesmal mit Hinweis im User- oder Systemprompt,
            # dass jetzt eine finale Antwort formuliert werden soll.
            user_input = "Formuliere jetzt bitte eine finale Antwort für den Nutzer basierend auf allen bisherigen Tool-Ergebnissen."
            continue

    # Fallback, falls max_iterations erreicht
    return "Ich konnte trotz mehrerer Planungsrunden keine finale Antwort erzeugen."

## MCP-Tool-Invoker für den Agenten: agentic_call_mcp_tool

### Funktion
- Standardisierte Funktion, die der Agent für jedes Tool verwendet:
        - baut HTTP-Client,
        - initialisiert Session,
        - ruft session.call_tool(tool_name, args) auf,
        - normalisiert das Ergebnis in ein konsistentes Dict.

### Input
- tool_name: str – z. B. "geocode".
- args: dict – aufgelöste Argumente.
- MCP_URL – globaler MCP-Endpoint.

### Output
- dict:
    - {"isError": bool, "data": ...}.
    - data ist entweder:
        - raw_result.structuredContent (wenn vorhanden),
        - oder zusammengefasster Text aus raw_result.content.
- Debug-Prints:
    - Toolname + Args,
    - Typ und Repräsentation des rohen Ergebnisses.

In [56]:
from mcp.types import TextContent  # ggf. anpassen

async def agentic_call_mcp_tool(tool_name: str, args: dict) -> dict:
    print(f"[DEBUG] agentic_call_mcp_tool: {tool_name}({args})")
    async with streamable_http_client(MCP_URL) as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            raw_result = await session.call_tool(tool_name, args)
            print(f"[DEBUG] Raw tool result type: {type(raw_result)}")
            print(f"[DEBUG] Raw tool result repr: {repr(raw_result)[:400]}")

            # Normalisieren
            result_dict: dict = {
                "isError": getattr(raw_result, "isError", False),
            }

            # strukturierte Inhalte bevorzugen
            if getattr(raw_result, "structuredContent", None) is not None:
                result_dict["data"] = raw_result.structuredContent
            else:
                # Fallback: Text aus content zusammenbauen
                texts = []
                for c in getattr(raw_result, "content", []) or []:
                    if isinstance(c, TextContent):
                        texts.append(c.text)
                result_dict["data"] = "\n".join(texts)

            return result_dict

## Vollagentische Demo 1: Wetter heute in Barcelona

### Funktion
- Läuft den kompletten Agentenloop einmal durch:
    - User: „Wie ist heute (YYYY-MM-DD) das Wetter in Barcelona?“
    - Agent:
        - plant Toolkette (geocode → get_weather),
        - ruft Tools über agentic_call_mcp_tool aus,
        - aktualisiert Plan/History iterativ,
        - liefert finale Antwort an „Max Mustermann“.

### Input
- history_agentic: Start-History:
    - Userinstruktion zur Anrede („Sprich mich bitte … Max Mustermann …“).
- user_input: Wetterfrage mit Datum.
- agent_system_prompt, agentic_call_mcp_tool.

### Output
- final_answer: str – fertige Reise/Wetter-Antwort, die:
    - das Wetter für Barcelona beschreibt,
    - Tools und echte Daten genutzt hat,
    - die Anrede „Max Mustermann“ respektiert.

### Konsole:
- „Finale Antwort an den User:“ + Text.

In [57]:
#history_agentic = [
#    {
#        "role": "user",
#        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
#    },
#    # weitere Messages:
#    # {"role": "assistant", "content": "..."},
#]

user_input = f"Ich möchte eine Reise nach Barcelona machen. Wie ist heute ({today}) das Wetter in Barcelona?"
history_agentic = []

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


iteration: 0, prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\n    Du bist ein persönlicher Reiseplaner.\n\n    Deine Aufgaben:\n    - Beantworte Fragen zu Urlaub, Reisen, Städten, Sehenswürdigkeiten, Aktivitäten, Regionalkultur oder Reiserouten.\n    - Du arbeitest in einem Szenario mit zusätzlichem Kontext, der dir vom System bereitgestellt wird (z.B. als „Kontext (aus Retrieval)“).\n    - Dir stehen MCP-Tools zur Verfügung, um Informationen zu beschaffen, die nicht in deinem Weltwissen vorhanden sind.\n\n    Nutzung von Kontext (RAG):\n    - Wenn dir ein Kontexttext vom System bereitgestellt wird, behandle ihn als zuverlässige Wissensquelle für diese Konversation.\n    - Du darfst alle darin enthaltenen Fakten verwenden.\n    - Wenn eine Information im Kontext steht, darfst du sie verwenden, auch wenn du sie nicht aus deinem allgemeinen Weltwissen kennst.\n\n    Allgemeines Arbeitsprinzip (ReAct mit Planung und Tool-Chains):\n    - Denke zuerst still über ei

[01/27/26 15:54:04] INFO     Created new          ]8;id=27975;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=429459;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             89a4f91a4f6c43ada79e                               
                             e2d6d10ac75e                                       
[01/27/26 15:54:05] INFO     Processing request of type            ]8;id=706510;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=897719;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py

iteration: 0, assistant_text: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona",\n          "name": "geocode",\n          "arguments": {\n            "destination": "Barcelona"\n          }\n        }\n      ]\n    },\n    {\n      "description": "Tageswetter für Barcelona am 27.01.2026 abrufen",\n      "tools": [\n        {\n          "id": "wetter_barcelona",\n          "name": "get_weather",\n          "arguments": {\n            "lat": { "$ref": "geo_barcelona.lat" },\n            "lon": { "$ref": "geo_barcelona.lon" },\n            "start_date": "2026-01-27",\n            "end_date": "2026-01-27",\n            "include_raw": false\n          }\n        }\n      ]\n    }\n  ],\n  "final": null\n}'
[DEBUG] raw output: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barc

[01/27/26 15:54:17] INFO     Created new          ]8;id=262871;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=354081;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/streamable_http_manager.py#239\239]8;;\
                             transport with                                     
                             session ID:                                        
                             0832b65d2daf41969535                               
                             2e168a285eca                                       
                    INFO     Processing request of type            ]8;id=785570;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=403975;file:///home/simon/.virtualenvs/KI-Workshop_2/lib/python3.10/site-packages/mcp/server/lowlevel/server.p

iteration: 1, assistant_text: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona",\n          "name": "geocode",\n          "arguments": {\n            "destination": "Barcelona"\n          }\n        }\n      ]\n    },\n    {\n      "description": "Tageswetter für Barcelona am 27.01.2026 abrufen",\n      "tools": [\n        {\n          "id": "wetter_barcelona",\n          "name": "get_weather",\n          "arguments": {\n            "lat": { "$ref": "geo_barcelona.lat" },\n            "lon": { "$ref": "geo_barcelona.lon" },\n            "start_date": "2026-01-27",\n            "end_date": "2026-01-27",\n            "include_raw": false\n          }\n        }\n      ]\n    }\n  ],\n  "final": null\n}'
[DEBUG] raw output: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barc

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


iteration: 2, assistant_text: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona",\n          "name": "geocode",\n          "arguments": {\n            "destination": "Barcelona"\n          }\n        }\n      ]\n    },\n    {\n      "description": "Wetter für Barcelona abrufen",\n      "tools": [\n        {\n          "id": "wetter_barcelona",\n          "name": "get_weather",\n          "arguments": {\n            "lat": { "$ref": "geo_barcelona.lat" },\n            "lon": { "$ref": "geo_barcelona.lon" },\n            "start_date": "2026-01-27",\n            "end_date": "2026-01-27",\n            "include_raw": false\n          }\n        }\n      ]\n    }\n  ],\n  "final": null\n}'
[DEBUG] raw output: 'assistant\n\n{\n  "steps": [\n    {\n      "description": "Geokoordinaten für Barcelona bestimmen",\n      "tools": [\n        {\n          "id": "geo_barcelona",\n          

## Vollagentische Demo 2: Beste Jogging-Spots um Barcelona

### Funktion
- Zeigt, wie der Agent mehrstufige Tool-Chains planen kann:
    - Geocoding von Barcelona,
    - Parks in Barcekona,
    - Abstand der Parks zum Zentrum.

### Input
- history_agentic: wieder mit Anrede-Instruktion.
- user_input: „Was sind die besten Plätze im Umkreis von 5km rund um Barcelona um zu joggen?“
- Rest wie oben.

### Output
- final_answer: str – Empfehlungsliste von Jogging-Spots:
- sortiert nach Score/Distanz

### Intern:
- Agent plant und ruft Tools wie geocode, get_spots oder rank_spots auf,
- verbindet die Ergebnisse über $ref.

In [ ]:
history_agentic = [
    {
        "role": "user",
        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
    },
    # weitere Messages:
    # {"role": "assistant", "content": "..."},
]

user_input = f"Was sind die besten Plätze im Umkreis von 5km rund um Barcelona um zu joggen?"

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)

## Vollagentische Demo 3: RAG + Tools mit Fantasietier Razepato

### Funktion
- Vereint alle Bausteine:
    - RAG-Wissen zum Fantasietier Razepato (aus Teil 2),
    - Geodaten / Weather / Spots via MCP,
    - Agentenplanung und Tool-Ketten.
- Beispiel:
    - User: „Ich möchte ein Razepato in Barcelona sehen. Wann und wohin muss ich reisen?“
    - Agent:
        - nutzt RAG-Kontext zum Razepato:
            - Lebensraum, Tageszeit, Jahreszeit,
        - identifiziert geeignete Regionen/Plätze (ggf. Parks),
        - kombiniert das mit Wetter/Season-Daten,
        - beantwortet die Frage im Reiseplan-Stil.

### Input
- history_agentic: mit der Namensinstruktion.
- user_input: Razepato-Reisewunsch.
- agent_system_prompt, RAG-Funktionen (via Prompt-Builder), MCP-Tools.

### Output
- final_answer: str – eine Agentenantwort, die:
    - auf dem RAG-Text über das Razepato basiert,
    - realistische Orte/Zeitfenster in Barcelona nennt (innerhalb des fiktiven Kontextes),
    - Tools dort nutzt, wo echte Daten nötig sind (z. B. Wetter),
    - wieder „Max Mustermann“ richtig anspricht.

In [ ]:
history_agentic = [
    {
        "role": "user",
        "content": "Hi, mein Name ist Max Mustermann. Sprich mich bitte in jeder Antwort mit meinem Namen an. Sag explizit dazu, dass du mich in jeder Antwort mit meinem Namen ansprechen wirst."
    },
    # weitere Messages:
    # {"role": "assistant", "content": "..."},
]

user_input = f"Ich möchte ein Razepato in Barcelona sehen. Wann und wohin muss ich reisen?"

final_answer = await agentic_run_to_final_answer(
    user_input=user_input,
    system_prompt=agent_system_prompt,
    history=history_agentic,
    tool_history=None,
    call_tool_fn=agentic_call_mcp_tool,          # deine bestehende MCP-Tool-Invoker-Funktion
    build_prompt_fn=agentic_build_chat_prompt_with_rag_and_tools,
)

print("Finale Antwort an den User:")
print(final_answer)